Merging Data

In [157]:
import os
import pandas as pd
import glob
from pathlib import Path

def process_commodity_data(input_folder, output_folder):
    """
    Process commodity production data from Excel files and create merged outputs.
    
    Args:
        input_folder (str): Path to the folder containing raw commodity data
        output_folder (str): Path to the folder where merged files will be saved
    """
    
    # Get a list of all commodity folders
    commodity_folders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))]
    
    for commodity in commodity_folders:
        print(f"Processing {commodity}...")
        commodity_path = os.path.join(input_folder, commodity)
        
        # Get all Excel files for this commodity
        excel_files = sorted(glob.glob(os.path.join(commodity_path, f"ComProd-*-{commodity}.xlsx")))
        
        # Sort files in chronological order (newest first to oldest last)
        excel_files = sorted(excel_files, key=lambda x: int(x.split('-')[1]), reverse=True)
        
        # Read and process each file
        df_list = []
        for file_path in excel_files:
            df = process_excel_file(file_path)
            df_list.append(df)
        
        # Merge the dataframes
        merged_df = merge_dataframes(df_list)
        
        # Sort columns chronologically
        merged_df = sort_columns_chronologically(merged_df)
        
        # Save the merged data with the specified naming convention
        output_path = os.path.join(output_folder, f"{commodity}-Production-1980-2023.xlsx")
        merged_df.to_excel(output_path, index=False)
        
        # Remove empty row after header in the output file
        remove_empty_row_after_header(output_path)
        
        print(f"Saved merged data to {output_path}")

def remove_empty_row_after_header(file_path):
    """
    Remove the empty row after the header in the Excel file.
    
    Args:
        file_path (str): Path to the Excel file
    """
    # Read the saved Excel file
    df = pd.read_excel(file_path)
    
    # Check if there's at least one row and if the first row is empty
    if len(df) > 0 and df.iloc[0].isna().all():
        # Remove the empty row
        df = df.drop(0).reset_index(drop=True)
        # Save the file again
        df.to_excel(file_path, index=False)
        print(f"Removed empty row after header in {file_path}")

def process_excel_file(file_path):
    """
    Process an individual Excel file:
    - Skip the first 4 rows
    - Remove the 7th row (which is now the 3rd row after skipping)
    - Set the appropriate headers
    - Fix year column names
    
    Args:
        file_path (str): Path to the Excel file
    
    Returns:
        pandas.DataFrame: Processed dataframe with standardized column names
    """
    # Read Excel file, skipping the first 4 rows
    df = pd.read_excel(file_path, skiprows=4)
    
    # Drop the empty row (originally the 7th row, now the 3rd after skipping 4)
    df = df.drop(2, axis=0).reset_index(drop=True)
    
    # First, detect the actual property column names
    first_col_name = df.columns[0]
    second_col_name = df.columns[1]
    
    # Get the year values from the first row
    years = df.iloc[0, 2:].astype(str)
    
    # Set initial column names preserving the original property column names
    df.columns = [first_col_name, second_col_name] + list(years)
    
    # Remove the first row (which contained the years)
    df = df.iloc[1:].reset_index(drop=True)
    
    # Standardize property column names
    column_mapping = {
        first_col_name: 'PROP_NAME',
        second_col_name: 'PROP_ID'
    }
    
    # Clean year column names and create new column names list
    new_cols = []
    for col in df.columns:
        if col in [first_col_name, second_col_name]:
            new_cols.append(column_mapping[col])
        else:
            # Remove 'Y' at the end
            new_col = col.replace('Y', '')
            
            # Handle different header formats
            if 'COMMODITY_PRODUCTION_TONNE_BY_PERIOD' in new_col:
                new_col = new_col.split('_')[-1]  # Extract just the year part
            elif any(term in new_col for term in ['TONNE', 'PRODUCTION']):
                # Generic handler for any production data columns
                parts = new_col.split('_')
                # Get the last part which is typically the year
                if parts and parts[-1].isdigit():
                    new_col = parts[-1]
            
            new_cols.append(new_col)
    
    df.columns = new_cols
    
    # Remove any empty rows
    df = df.dropna(how='all').reset_index(drop=True)
    
    return df

def merge_dataframes(df_list):
    """
    Merge multiple dataframes by keeping PROP_NAME and PROP_ID columns from the first dataframe
    and appending production data columns from all dataframes.
    
    Args:
        df_list (list): List of dataframes to merge
    
    Returns:
        pandas.DataFrame: Merged dataframe with no empty rows
    """
    if not df_list:
        return pd.DataFrame()
    
    # Verify all dataframes have the standardized property columns
    for i, df in enumerate(df_list):
        if 'PROP_NAME' not in df.columns or 'PROP_ID' not in df.columns:
            print(f"Warning: DataFrame {i} is missing standard property columns. Available columns: {df.columns.tolist()}")
            # This is a defensive measure to prevent errors, but should not be needed with the updated process_excel_file
            return pd.DataFrame()
    
    # Use the first dataframe as the base
    result_df = df_list[0][['PROP_NAME', 'PROP_ID']].copy()
    
    # Add production data columns from each dataframe
    for df in df_list:
        production_columns = [col for col in df.columns if col not in ['PROP_NAME', 'PROP_ID']]
        if production_columns:  # Only merge if there are production columns
            result_df = pd.merge(result_df, df[['PROP_ID'] + production_columns], on='PROP_ID', how='left')
    
    # Ensure there are no empty rows in the final result
    result_df = result_df.dropna(how='all').reset_index(drop=True)
    
    # Remove any rows that are completely empty (except for PROP_NAME and PROP_ID)
    if len(result_df) > 0:
        # Check if a row contains data in any column other than PROP_NAME and PROP_ID
        data_columns = [col for col in result_df.columns if col not in ['PROP_NAME', 'PROP_ID']]
        if data_columns:
            has_data = ~result_df[data_columns].isna().all(axis=1)
            result_df = result_df[has_data].reset_index(drop=True)
    
    return result_df

def sort_columns_chronologically(df):
    """
    Sort the production columns in chronological order (1980-2023).
    
    Args:
        df (pandas.DataFrame): Dataframe with production data
    
    Returns:
        pandas.DataFrame: Dataframe with chronologically sorted columns
    """
    # Separate identification columns and year columns
    id_columns = ['PROP_NAME', 'PROP_ID']
    year_columns = [col for col in df.columns if col not in id_columns]
    
    # Sort year columns
    year_columns.sort(key=lambda x: int(x) if x.isdigit() else 0)
    
    # Reorder columns
    sorted_df = df[id_columns + year_columns]
    
    return sorted_df

def main():
    """
    Main function to execute the data processing pipeline.
    """
    try:
        input_folder = "Data Input/Commodity Production_Raw"
        output_folder = "Data Input/Commodity Production"
        process_commodity_data(input_folder, output_folder)
        print("All commodities processed successfully!")
    except Exception as e:
        print(f"Error processing commodity data: {e}")

if __name__ == "__main__":
    main()

Processing Bauxite...
Saved merged data to Data Input/Commodity Production\Bauxite-Production-1980-2023.xlsx
Processing Copper...
Saved merged data to Data Input/Commodity Production\Copper-Production-1980-2023.xlsx
Processing IronOre...
Saved merged data to Data Input/Commodity Production\IronOre-Production-1980-2023.xlsx
Processing Lithium...
Saved merged data to Data Input/Commodity Production\Lithium-Production-1980-2023.xlsx
Processing Manganese...
Saved merged data to Data Input/Commodity Production\Manganese-Production-1980-2023.xlsx
Processing Nickel...
Saved merged data to Data Input/Commodity Production\Nickel-Production-1980-2023.xlsx
Processing Zinc...
Saved merged data to Data Input/Commodity Production\Zinc-Production-1980-2023.xlsx
All commodities processed successfully!


In [158]:
import os
import pandas as pd
import glob

def merge_with_metadata(commodity_folder, metadata_file):
    """
    Merge all Excel files in the commodity folder with metadata file based on PROP_ID.
    
    Args:
        commodity_folder (str): Path to the folder containing commodity files
        metadata_file (str): Path to the metadata file
    """
    print(f"Starting metadata merge process...")
    
    # Check if metadata file exists
    if not os.path.exists(metadata_file):
        print(f"Error: Metadata file {metadata_file} not found.")
        return
    
    # Load the metadata file
    try:
        metadata_df = pd.read_excel(metadata_file)
        print(f"Loaded metadata file with {len(metadata_df)} rows.")
        
        # Verify metadata file has PROP_ID column
        if 'PROP_ID' not in metadata_df.columns:
            print(f"Error: Metadata file does not contain PROP_ID column.")
            return
    except Exception as e:
        print(f"Error loading metadata file: {e}")
        return
    
    # Get all commodity Excel files
    excel_files = glob.glob(os.path.join(commodity_folder, "*.xlsx"))
    
    if not excel_files:
        print(f"No Excel files found in {commodity_folder}")
        return
    
    print(f"Found {len(excel_files)} Excel files to process.")
    
    # Process each Excel file
    for file_path in excel_files:
        file_name = os.path.basename(file_path)
        print(f"Processing {file_name}...")
        
        try:
            # Load the commodity file
            commodity_df = pd.read_excel(file_path)
            
            # Verify commodity file has PROP_ID column
            if 'PROP_ID' not in commodity_df.columns:
                print(f"Warning: File {file_name} does not contain PROP_ID column. Skipping...")
                continue
            
            # Identify production columns (year columns from 1980-2023)
            prod_columns = []
            for col in commodity_df.columns:
                # Check if column name is a year between 1980 and 2023
                try:
                    year = int(col)
                    if 1980 <= year <= 2023:
                        prod_columns.append(col)
                except (ValueError, TypeError):
                    # Not a year column
                    pass
            
            # Identify metadata columns (exclude duplicated columns)
            metadata_columns = [col for col in metadata_df.columns 
                               if col != 'PROP_NAME']  # PROP_ID will be used for merge
            
            # Merge with metadata on PROP_ID, keeping only matching rows
            merged_df = pd.merge(commodity_df, metadata_df, on='PROP_ID', how='inner')
            
            # Check if any rows were matched
            if len(merged_df) == 0:
                print(f"Warning: No matching PROP_IDs found in {file_name}. File will be empty.")
            else:
                print(f"Merged {file_name} - {len(merged_df)} matching rows found.")
            
            # Remove duplicate PROP_NAME column if it exists in both files
            if 'PROP_NAME_x' in merged_df.columns and 'PROP_NAME_y' in merged_df.columns:
                # Keep the metadata version (usually PROP_NAME_y)
                merged_df = merged_df.drop('PROP_NAME_x', axis=1)
                merged_df = merged_df.rename(columns={'PROP_NAME_y': 'PROP_NAME'})
            
            # Reorder columns: metadata columns first, then production columns
            # Start with PROP_ID and PROP_NAME
            final_columns = ['PROP_ID', 'PROP_NAME']
            
            # Add other metadata columns (that aren't already in final_columns)
            other_metadata = [col for col in metadata_df.columns 
                             if col not in final_columns and col != 'PROP_NAME_y']
            final_columns.extend(other_metadata)
            
            # Add production columns at the end
            final_columns.extend([col for col in prod_columns if col not in final_columns])
            
            # Filter to include only columns that exist in merged_df
            final_columns = [col for col in final_columns if col in merged_df.columns]
            
            # Reorder the dataframe
            merged_df = merged_df[final_columns]
            
            # Save back to the original file
            merged_df.to_excel(file_path, index=False)
            print(f"Saved merged data back to {file_name}")
            
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
    
    print("Metadata merge process completed.")

def main():
    """
    Main function to execute the metadata merge process.
    """
    try:
        commodity_folder = "Data Input/Commodity Production"
        metadata_file = "Data Input/Meta Data/Meta_Data_Columns_final.xlsx"
        merge_with_metadata(commodity_folder, metadata_file)
    except Exception as e:
        print(f"Error in metadata merge process: {e}")

if __name__ == "__main__":
    main()

Starting metadata merge process...
Loaded metadata file with 34843 rows.
Found 7 Excel files to process.
Processing Bauxite-Production-1980-2023.xlsx...
Merged Bauxite-Production-1980-2023.xlsx - 73 matching rows found.
Saved merged data back to Bauxite-Production-1980-2023.xlsx
Processing Copper-Production-1980-2023.xlsx...
Merged Copper-Production-1980-2023.xlsx - 888 matching rows found.
Saved merged data back to Copper-Production-1980-2023.xlsx
Processing IronOre-Production-1980-2023.xlsx...
Merged IronOre-Production-1980-2023.xlsx - 716 matching rows found.
Saved merged data back to IronOre-Production-1980-2023.xlsx
Processing Lithium-Production-1980-2023.xlsx...
Merged Lithium-Production-1980-2023.xlsx - 38 matching rows found.
Saved merged data back to Lithium-Production-1980-2023.xlsx
Processing Manganese-Production-1980-2023.xlsx...
Merged Manganese-Production-1980-2023.xlsx - 37 matching rows found.
Saved merged data back to Manganese-Production-1980-2023.xlsx
Processing Nick

Processing Data

In [159]:
import os
import pandas as pd
import glob

def process_commodity_files(folder_path, output_folder):
    """
    Process all Excel files in a folder, removing rows where the PRIMARY_COMMODITY
    doesn't match the commodity indicated in the filename.
   
    Args:
        folder_path: Path to the folder containing Excel files
        output_folder: Path to save extracted mismatched rows
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
   
    # Get all Excel files in the folder
    excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
   
    # Process each Excel file
    for file_path in excel_files:
        # Skip the merged file if it exists in the same folder
        if "All_Commodities" in file_path:
            continue
           
        filename = os.path.basename(file_path)
        print(f"\nProcessing file: {filename}")
       
        # Extract the commodity name from the filename
        # Format: [primary commodity]-Production-1980-2023.xlsx
        commodity_name = filename.split('-')[0]
       
        try:
            # Read the Excel file
            df = pd.read_excel(file_path)
           
            # Check if required columns exist
            if 'PRIMARY_COMMODITY' not in df.columns:
                print(f"Warning: PRIMARY_COMMODITY column not found in {filename}")
                continue
               
            if 'PROP_ID' not in df.columns:
                print(f"Warning: PROP_ID column not found in {filename}")
                continue
           
            # Get initial row count
            initial_count = len(df)
           
            # Handle special cases
            if commodity_name == "IronOre":
                expected_commodity = "Iron Ore"
            else:
                expected_commodity = commodity_name
           
            # Find rows where PRIMARY_COMMODITY doesn't match the expected value
            mismatched_rows = df[df['PRIMARY_COMMODITY'] != expected_commodity]
           
            # Add filename column to mismatched rows for reference
            mismatched_rows['SOURCE_FILE'] = filename
           
            # Keep only the matching rows
            df_filtered = df[df['PRIMARY_COMMODITY'] == expected_commodity]
           
            # Count removed rows
            removed_count = initial_count - len(df_filtered)
           
            # Save the filtered data back to the original file
            df_filtered.to_excel(file_path, index=False)
           
            # Save mismatched rows to individual file in the output folder
            if not mismatched_rows.empty:
                # Create output filename based on the original filename
                output_filename = os.path.join(output_folder, f"{commodity_name}_mismatched.xlsx")
                mismatched_rows.to_excel(output_filename, index=False)
               
                print(f"Removed {removed_count} rows with mismatched commodity type")
                print(f"Remaining rows: {len(df_filtered)}")
                print(f"File saved back to: {file_path}")
                print(f"Mismatched rows saved to: {output_filename}")
            else:
                print(f"No mismatched rows found in {filename}")
           
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

if __name__ == "__main__":
    # Define folder paths
    input_folder = "Data Input/Commodity Production"
    output_folder = "Data Input/Commodity Production/Secondary"
   
    # Run the process
    process_commodity_files(input_folder, output_folder)


Processing file: Bauxite-Production-1980-2023.xlsx
No mismatched rows found in Bauxite-Production-1980-2023.xlsx

Processing file: Copper-Production-1980-2023.xlsx


C:\Users\leshe\AppData\Local\Temp\ipykernel_12532\1813254057.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatched_rows['SOURCE_FILE'] = filename


Removed 329 rows with mismatched commodity type
Remaining rows: 559
File saved back to: Data Input/Commodity Production\Copper-Production-1980-2023.xlsx
Mismatched rows saved to: Data Input/Commodity Production/Secondary\Copper_mismatched.xlsx

Processing file: IronOre-Production-1980-2023.xlsx


C:\Users\leshe\AppData\Local\Temp\ipykernel_12532\1813254057.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatched_rows['SOURCE_FILE'] = filename


Removed 16 rows with mismatched commodity type
Remaining rows: 700
File saved back to: Data Input/Commodity Production\IronOre-Production-1980-2023.xlsx
Mismatched rows saved to: Data Input/Commodity Production/Secondary\IronOre_mismatched.xlsx

Processing file: Lithium-Production-1980-2023.xlsx
Removed 1 rows with mismatched commodity type
Remaining rows: 37
File saved back to: Data Input/Commodity Production\Lithium-Production-1980-2023.xlsx
Mismatched rows saved to: Data Input/Commodity Production/Secondary\Lithium_mismatched.xlsx

Processing file: Manganese-Production-1980-2023.xlsx


...

Removed 3 rows with mismatched commodity type
Remaining rows: 34
File saved back to: Data Input/Commodity Production\Manganese-Production-1980-2023.xlsx
Mismatched rows saved to: Data Input/Commodity Production/Secondary\Manganese_mismatched.xlsx

Processing file: Nickel-Production-1980-2023.xlsx


C:\Users\leshe\AppData\Local\Temp\ipykernel_12532\1813254057.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatched_rows['SOURCE_FILE'] = filename


Removed 261 rows with mismatched commodity type
Remaining rows: 140
File saved back to: Data Input/Commodity Production\Nickel-Production-1980-2023.xlsx
Mismatched rows saved to: Data Input/Commodity Production/Secondary\Nickel_mismatched.xlsx

Processing file: Zinc-Production-1980-2023.xlsx


C:\Users\leshe\AppData\Local\Temp\ipykernel_12532\1813254057.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatched_rows['SOURCE_FILE'] = filename


Removed 229 rows with mismatched commodity type
Remaining rows: 305
File saved back to: Data Input/Commodity Production\Zinc-Production-1980-2023.xlsx
Mismatched rows saved to: Data Input/Commodity Production/Secondary\Zinc_mismatched.xlsx


In [160]:
import os
import pandas as pd
import glob
import numpy as np

def remove_zero_rows(folder_path, min_nonzero_values):
    """
    Process all Excel files in the specified folder and remove rows where:
    1. All year columns (1980-2023) contain only zeros or empty values, OR
    2. The row has fewer than the minimum required non-zero production values
   
    Args:
        folder_path (str): Path to the folder containing Excel files
        min_nonzero_values (int): Minimum number of non-zero values required to keep a row
    """
    print(f"Starting to process files in {folder_path}...")
    print(f"Minimum non-zero values required: {min_nonzero_values}")
   
    # Get all Excel files in the folder
    excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
   
    if not excel_files:
        print(f"No Excel files found in {folder_path}")
        return
   
    print(f"Found {len(excel_files)} Excel files to process.")
   
    # Process each Excel file
    for file_path in excel_files:
        file_name = os.path.basename(file_path)
        print(f"Processing {file_name}...")
       
        try:
            # Load the file
            df = pd.read_excel(file_path)
            initial_rows = len(df)
           
            # Identify year columns (1980-2023)
            year_columns = []
            for col in df.columns:
                try:
                    year = int(col)
                    if 1980 <= year <= 2023:
                        year_columns.append(col)
                except (ValueError, TypeError):
                    # Not a year column
                    pass
           
            if not year_columns:
                print(f"Warning: No year columns found in {file_name}. Skipping file.")
                continue
           
            print(f"Found {len(year_columns)} year columns in {file_name}.")
           
            # Count non-zero, non-empty values for each row
            nonzero_counts = df[year_columns].apply(lambda row:
                sum(
                    1 for value in row 
                    if (not pd.isna(value)) and (value != 0) and (value != '0') and (value != '')
                ),
                axis=1
            )
           
            # Create a mask to identify rows that meet the minimum threshold
            meets_threshold = nonzero_counts >= min_nonzero_values
           
            # Keep only rows that meet the minimum threshold
            df_filtered = df[meets_threshold].reset_index(drop=True)
           
            # Calculate how many rows were removed
            removed_rows = initial_rows - len(df_filtered)
            
            # Show breakdown of removed rows
            zero_rows = sum(nonzero_counts == 0)
            insufficient_rows = sum((nonzero_counts > 0) & (nonzero_counts < min_nonzero_values))
            
            print(f"Removed {removed_rows} rows from {file_name}:")
            print(f"  - Rows with zero production: {zero_rows}")
            print(f"  - Rows with insufficient production (<{min_nonzero_values} non-zero values): {insufficient_rows}")
            print(f"  - Remaining rows: {len(df_filtered)}")
           
            # Save the filtered data back to the original file
            df_filtered.to_excel(file_path, index=False)
            print(f"Saved filtered data back to {file_name}")
           
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
   
    print("Row removal process completed.")

def main():
    """
    Main function to execute the row removal process.
    """
    try:
        folder_path = "Data Input/Commodity Production"
        # Set minimum non-zero values required (default is 5)
        min_nonzero_threshold = 5
        remove_zero_rows(folder_path, min_nonzero_threshold)
    except Exception as e:
        print(f"Error in row removal process: {e}")

if __name__ == "__main__":
    main()

Starting to process files in Data Input/Commodity Production...
Minimum non-zero values required: 5
Found 7 Excel files to process.
Processing Bauxite-Production-1980-2023.xlsx...
Found 44 year columns in Bauxite-Production-1980-2023.xlsx.
Removed 10 rows from Bauxite-Production-1980-2023.xlsx:
  - Rows with zero production: 0
  - Rows with insufficient production (<5 non-zero values): 10
  - Remaining rows: 63
Saved filtered data back to Bauxite-Production-1980-2023.xlsx
Processing Copper-Production-1980-2023.xlsx...
Found 44 year columns in Copper-Production-1980-2023.xlsx.
Removed 181 rows from Copper-Production-1980-2023.xlsx:
  - Rows with zero production: 71
  - Rows with insufficient production (<5 non-zero values): 110
  - Remaining rows: 378
Saved filtered data back to Copper-Production-1980-2023.xlsx
Processing IronOre-Production-1980-2023.xlsx...
Found 44 year columns in IronOre-Production-1980-2023.xlsx.
Removed 182 rows from IronOre-Production-1980-2023.xlsx:
  - Rows with

In [161]:
import os
import pandas as pd

def process_xlsx_files(input_folder, output_folder):
    """
    Process all xlsx files in the input folder by:
    1. Removing rows with specified values in DEV_STAGE column
    2. Removing rows with specified values in MINE_TYPE1 column
    3. Removing rows with empty/null values in MINE_TYPE1 column
    4. Removing rows that have 'Closed' in DEV_STAGE AND ('Inactive' OR 'Care and Maintenance') in ACTV_STATUS
   
    Args:
        input_folder (str): Path to folder with input xlsx files
        output_folder (str): Path to save processed xlsx files (not used, as we're saving to the same file)
    """
    # List of DEV_STAGE values to filter out
    dev_stages_to_remove = [
        'Grassroots',
        'Exploration',
        'Advanced Exploration',
        'Target Outline',
        'Reserves Development',
        'Construction Planned',
        'Construction Started',
        'Prefeas/Scoping',
        'Feasibility Started',
        'Feasibility Complete',
        'Feasibility'
    ]
   
    # List of MINE_TYPE1 values to filter out
    mine_types_to_remove = [
        'Dredging',
        'Dump',
        'Tailings',
        'Stock Pile',
        'Placer',
        'Ocean',
        'In-Situ Leach'
    ]
   
    # Get all xlsx files in the input folder
    try:
        files = [f for f in os.listdir(input_folder) if f.endswith('.xlsx')]
    except FileNotFoundError:
        print(f"Error: Input folder '{input_folder}' not found.")
        return
   
    if not files:
        print(f"No xlsx files found in '{input_folder}'.")
        return
   
    # Process each file
    for file in files:
        input_path = os.path.join(input_folder, file)
       
        print(f"Processing {file}...")
       
        try:
            # Read the Excel file
            df = pd.read_excel(input_path)
            original_rows = len(df)
           
            # Check if required columns exist
            required_columns = ['DEV_STAGE', 'START_UP_YR', 'MINE_TYPE1', 'ACTV_STATUS']
            missing_columns = [col for col in required_columns if col not in df.columns]
           
            if missing_columns:
                print(f"Warning: Required columns {missing_columns} not found in {file}, skipping.")
                continue
           
            # Filter out rows with specified DEV_STAGE values
            df_filtered_stage = df[~df['DEV_STAGE'].isin(dev_stages_to_remove)]
           
            # Filter out rows with specified MINE_TYPE1 values
            df_filtered_minetype = df_filtered_stage[~df_filtered_stage['MINE_TYPE1'].isin(mine_types_to_remove)]
            
            # Filter out rows with empty/null MINE_TYPE1 values
            df_filtered_minetype_nonempty = df_filtered_minetype[df_filtered_minetype['MINE_TYPE1'].notna() & (df_filtered_minetype['MINE_TYPE1'] != '')]
            
            # Filter out rows that have 'Closed' in DEV_STAGE AND ('Inactive' OR 'Care and Maintenance') in ACTV_STATUS
            closed_and_inactive_mask = (df_filtered_minetype_nonempty['DEV_STAGE'] == 'Closed') & (
                (df_filtered_minetype_nonempty['ACTV_STATUS'] == 'Inactive') | 
                (df_filtered_minetype_nonempty['ACTV_STATUS'] == 'Care and Maintenance')
            )
            df_filtered = df_filtered_minetype_nonempty[~closed_and_inactive_mask]
           
            # Save the filtered DataFrame back to the same Excel file
            df_filtered.to_excel(input_path, index=False)
           
            # Report results
            rows_removed_stage = original_rows - len(df_filtered_stage)
            rows_removed_mine_type = len(df_filtered_stage) - len(df_filtered_minetype)
            rows_removed_empty_minetype = len(df_filtered_minetype) - len(df_filtered_minetype_nonempty)
            rows_removed_closed_inactive = len(df_filtered_minetype_nonempty) - len(df_filtered)
            total_rows_removed = original_rows - len(df_filtered)
           
            print(f"  Updated {file}:")
            print(f"    - Removed {rows_removed_stage} rows due to development stage")
            print(f"    - Removed {rows_removed_mine_type} rows due to mine type")
            print(f"    - Removed {rows_removed_empty_minetype} rows due to empty mine type")
            print(f"    - Removed {rows_removed_closed_inactive} rows due to Closed + (Inactive or Care and Maintenance) combination")
            print(f"    - Total removed: {total_rows_removed} rows")
            print(f"    - Remaining rows: {len(df_filtered)}")
           
        except Exception as e:
            print(f"Error processing {file}: {e}")
   
    print("Processing complete.")

# Example usage
if __name__ == "__main__":
    input_folder = "Data Input/Commodity Production"
    output_folder = "Data Input/Commodity Production"  # Not used but kept for compatibility
    process_xlsx_files(input_folder, output_folder)

Processing Bauxite-Production-1980-2023.xlsx...
  Updated Bauxite-Production-1980-2023.xlsx:
    - Removed 0 rows due to development stage
    - Removed 0 rows due to mine type
    - Removed 1 rows due to empty mine type
    - Removed 4 rows due to Closed + (Inactive or Care and Maintenance) combination
    - Total removed: 5 rows
    - Remaining rows: 58
Processing Copper-Production-1980-2023.xlsx...
  Updated Copper-Production-1980-2023.xlsx:
    - Removed 25 rows due to development stage
    - Removed 3 rows due to mine type
    - Removed 2 rows due to empty mine type
    - Removed 35 rows due to Closed + (Inactive or Care and Maintenance) combination
    - Total removed: 65 rows
    - Remaining rows: 313
Processing IronOre-Production-1980-2023.xlsx...
  Updated IronOre-Production-1980-2023.xlsx:
    - Removed 9 rows due to development stage
    - Removed 5 rows due to mine type
    - Removed 50 rows due to empty mine type
    - Removed 31 rows due to Closed + (Inactive or Care and 

In [162]:
import os
import pandas as pd

def merge_main_commodity_files():
    """
    Merges all xlsx files in the main commodity folder
   
    Returns:
        str: Path to the merged commodities file
    """
    # Define paths
    main_folder = "Data Input/Commodity Production"
    merged_folder = "Data Output/Commodity Production"
   
    # Ensure the output directory exists
    os.makedirs(merged_folder, exist_ok=True)
   
    # Define output file
    all_commodities_file = os.path.join(merged_folder, "Commodity_Production-1980_2023.xlsx")
   
    print("Merging commodity files...")
   
    # Initialize list to store all dataframes
    all_dfs = []
   
    # Process main folder files
    main_files = [f for f in os.listdir(main_folder) if f.endswith('.xlsx')]
    main_file_count = 0
   
    for file in main_files:
        file_path = os.path.join(main_folder, file)
        # Skip directories (like subdirectories)
        if os.path.isdir(file_path):
            continue
           
        try:
            print(f"Reading file: {file}...")
            df = pd.read_excel(file_path)
            all_dfs.append(df)
            main_file_count += 1
        except Exception as e:
            print(f"Error processing {file}: {e}")
       
    # Merge all dataframes if we have any
    if all_dfs:
        merged_main = pd.concat(all_dfs, ignore_index=True)
        merged_main.to_excel(all_commodities_file, index=False)
        print(f"Successfully merged {main_file_count} commodity files "
              f"with {len(merged_main)} total rows.")
    else:
        print("No valid files to merge.")
   
    return all_commodities_file

if __name__ == "__main__":
    # Run the merge operation for commodity files
    commodities_file = merge_main_commodity_files()
   
    print("\nMerging operation complete.")

Merging commodity files...
Reading file: Bauxite-Production-1980-2023.xlsx...
Reading file: Copper-Production-1980-2023.xlsx...
Reading file: IronOre-Production-1980-2023.xlsx...
Reading file: Lithium-Production-1980-2023.xlsx...
Reading file: Manganese-Production-1980-2023.xlsx...
Reading file: Nickel-Production-1980-2023.xlsx...
Reading file: Zinc-Production-1980-2023.xlsx...
Successfully merged 7 commodity files with 1085 total rows.

Merging operation complete.


Processing part 
    1) Merge reserves data
    2) convert iron ore, manganese ore, to metal content and merge ore grade column into dataset
    3) literature review based start up year imputation --> there are 76 --> DO THIS FIRST
    4) literature review based closure year imputaiton --> DO THIS FIRST --> it is done for mines where there is production after the closure year
    5) if there is a non-zero production value before the start up year, change start up year to that year
    6) If there is a non-zero proudction value after projected closure year, change closure year to that year --> still doing this
    7) if start up year has 0 as production change it to the first year with non-zero production
    8) conduct hieararchical median based imputation for projected closure year of mines with missing closure years

In [163]:
import pandas as pd
import numpy as np
from pathlib import Path

def merge_reserves_to_production():
    """
    Merges reserves data to production data based on PROP_ID.
    Finds the highest reserves value per mine across all years (1980-2023).
    """
    
    # File paths
    reserves_file = "Data Input/Reserves/Reserves_Ore_Tonnage/Reserves_Tonnage-1980_2023.xlsx"
    production_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
    
    try:
        # Read the reserves file
        print("Reading reserves file...")
        reserves_df = pd.read_excel(reserves_file)
        print(f"Reserves file loaded: {reserves_df.shape[0]} rows, {reserves_df.shape[1]} columns")
        
        # Read the production file
        print("Reading production file...")
        production_df = pd.read_excel(production_file)
        print(f"Production file loaded: {production_df.shape[0]} rows, {production_df.shape[1]} columns")
        
        # Display column names for verification
        print("\nReserves file columns:")
        print(reserves_df.columns.tolist())
        print("\nProduction file columns:")
        print(production_df.columns.tolist())
        
        # Check if PROP_ID exists in both files
        if 'PROP_ID' not in reserves_df.columns:
            raise ValueError("PROP_ID column not found in reserves file")
        if 'PROP_ID' not in production_df.columns:
            raise ValueError("PROP_ID column not found in production file")
        
        # Identify year columns (1980-2023) in reserves data
        year_columns = []
        for col in reserves_df.columns:
            # Check if column name is a year between 1980-2023
            try:
                year = int(str(col))
                if 1980 <= year <= 2023:
                    year_columns.append(col)
            except (ValueError, TypeError):
                continue
        
        print(f"\nFound {len(year_columns)} year columns: {year_columns[:5]}..." if len(year_columns) > 5 else f"\nFound year columns: {year_columns}")
        
        if not year_columns:
            print("Warning: No year columns found. Checking for columns containing years...")
            # Alternative approach - look for columns that might contain year data
            year_columns = [col for col in reserves_df.columns 
                          if any(str(year) in str(col) for year in range(1980, 2024))]
            print(f"Alternative year columns found: {year_columns}")
        
        # Calculate maximum reserves value for each PROP_ID
        print("\nCalculating maximum reserves per PROP_ID...")
        
        # Create a copy of reserves data for processing
        reserves_work = reserves_df.copy()
        
        # Convert year columns to numeric, replacing non-numeric values with NaN
        for col in year_columns:
            reserves_work[col] = pd.to_numeric(reserves_work[col], errors='coerce')
        
        # Calculate max reserves for each PROP_ID across all years
        # First, calculate the maximum value across year columns for each row
        reserves_work['Reserves_Tonnage'] = reserves_work[year_columns].max(axis=1, skipna=True)
        
        # Filter out rows where the maximum value is 0, NaN, or negative
        reserves_work = reserves_work[
            (reserves_work['Reserves_Tonnage'] > 0) & 
            (reserves_work['Reserves_Tonnage'].notna()) &
            (reserves_work['Reserves_Tonnage'] != np.inf)
        ]
        
        # Then group by PROP_ID and take the maximum reserves value per property
        reserves_summary = reserves_work.groupby('PROP_ID')['Reserves_Tonnage'].max().reset_index()
        
        print(f"Calculated maximum reserves for {len(reserves_summary)} unique PROP_IDs")
        print(f"Sample reserves data:")
        print(reserves_summary.head())
        
        # Find the position to insert the new column (before START_UP_YR)
        if 'START_UP_YR' in production_df.columns:
            insert_position = production_df.columns.get_loc('START_UP_YR')
            print(f"\nWill insert Reserves_Tonnage column at position {insert_position} (before START_UP_YR)")
        else:
            print("\nSTART_UP_YR column not found. Will append Reserves_Tonnage at the end.")
            insert_position = len(production_df.columns)
        
        # Merge reserves data with production data
        print("\nMerging data...")
        merged_df = production_df.merge(reserves_summary, on='PROP_ID', how='left')
        
        # Reorder columns to place Reserves_Tonnage in the correct position
        columns = list(production_df.columns)
        if 'START_UP_YR' in columns:
            # Insert before START_UP_YR
            columns.insert(insert_position, 'Reserves_Tonnage')
            merged_df = merged_df[columns]
        
        print(f"Merge completed: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
        print(f"Records with reserves data: {merged_df['Reserves_Tonnage'].notna().sum()}")
        print(f"Records without reserves data: {merged_df['Reserves_Tonnage'].isna().sum()}")
        
        # Save the merged file
        output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
        print(f"\nSaving merged file to: {output_file}")
        
        # Create output directory if it doesn't exist
        Path(output_file).parent.mkdir(parents=True, exist_ok=True)
        
        merged_df.to_excel(output_file, index=False)
        print("File saved successfully!")
        
        # Display summary statistics
        print("\n" + "="*50)
        print("MERGE SUMMARY")
        print("="*50)
        print(f"Original production records: {len(production_df)}")
        print(f"Unique PROP_IDs in production: {production_df['PROP_ID'].nunique()}")
        print(f"Unique PROP_IDs in reserves: {reserves_df['PROP_ID'].nunique()}")
        print(f"Matched records with reserves: {merged_df['Reserves_Tonnage'].notna().sum()}")
        print(f"Unmatched records: {merged_df['Reserves_Tonnage'].isna().sum()}")
        
        if merged_df['Reserves_Tonnage'].notna().sum() > 0:
            print(f"\nReserves statistics:")
            print(f"Min reserves: {merged_df['Reserves_Tonnage'].min():,.0f}")
            print(f"Max reserves: {merged_df['Reserves_Tonnage'].max():,.0f}")
            print(f"Mean reserves: {merged_df['Reserves_Tonnage'].mean():,.0f}")
        
        return merged_df
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except Exception as e:
        print(f"Error during processing: {e}")
        raise

if __name__ == "__main__":
    print("Starting Excel file merge process...")
    print("="*50)
    
    try:
        result = merge_reserves_to_production()
        print("\nProcess completed successfully!")
    except Exception as e:
        print(f"\nProcess failed with error: {e}")

Starting Excel file merge process...
Reading reserves file...
Reserves file loaded: 9206 rows, 49 columns
Reading production file...
Production file loaded: 1085 rows, 57 columns

Reserves file columns:
['PROP_NAME_x', 'PROP_ID', 'PRIMARY_COMMODITY_x', 'PROP_NAME_y', 'PRIMARY_COMMODITY_y', 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

Production file columns:
['PROP_ID', 'PROP_NAME', 'PRIMARY_COMMODITY', 'COMMODITIES_LIST', 'COUNTRY_NAME', 'LATITUDE', 'LONGITUDE', 'DEV_STAGE', 'ACTV_STATUS', 'MINE_TYPE1', 'GEOLOGIC_ORE_BODY_TYPE', 'START_UP_YR', 'PROJ_CLOSURE_YR', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2

In [164]:
import pandas as pd

# File paths
reserves_file = "Data Input/Reserves/Reserves_Contained/Commodities_Reserves_Contained.xlsx"
commodity_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"

# Load data
reserves_df = pd.read_excel(reserves_file, engine="openpyxl")
commodity_df = pd.read_excel(commodity_file, engine="openpyxl")

# Keep only relevant columns from reserves
reserves_df = reserves_df[["PROP_ID", "Reserves_Contained"]]

# Merge on PROP_ID
merged_df = commodity_df.merge(reserves_df, on="PROP_ID", how="left")

# Reorder columns: place Reserves_Contained after Reserves_Tonnage
cols = list(merged_df.columns)
if "Reserves_Tonnage" in cols and "Reserves_Contained" in cols:
    idx = cols.index("Reserves_Tonnage") + 1
    # Remove and reinsert at the right place
    cols.insert(idx, cols.pop(cols.index("Reserves_Contained")))
    merged_df = merged_df[cols]

# Save result
merged_df.to_excel(output_file, index=False)
print(f"Saved merged file to {output_file}")

Saved merged file to Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx


In [165]:
import pandas as pd

# --- File paths ---
ore_grade_path = "Data Input/Ore Grade/Commodities_Ore_Grade.xlsx"
prod_path = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"

# --- Load data ---
grades = pd.read_excel(ore_grade_path, dtype={"PROP_ID": str})
prod = pd.read_excel(prod_path, dtype={"PROP_ID": str})

# Keep only needed cols (retain PRIMARY_COMMODITY if present so we can detect bauxite)
keep_cols = [c for c in ["PROP_ID", "Ore_Grade", "PRIMARY_COMMODITY"] if c in grades.columns]
grades = grades[keep_cols].drop_duplicates(subset=["PROP_ID"])

# If PRIMARY_COMMODITY not in ore-grade file, map it in from production (for tagging bauxite rows)
if "PRIMARY_COMMODITY" not in grades.columns:
    grades = grades.merge(
        prod[["PROP_ID", "PRIMARY_COMMODITY"]].drop_duplicates("PROP_ID"),
        on="PROP_ID",
        how="left"
    )

# --- Adjust bauxite grades in the ore-grade file: multiply by 0.529 BEFORE percent→decimal ---
# (Leaves defaults/fallbacks unchanged; only rows that already have a bauxite grade are adjusted.)
baux_mask_in_grades = grades["PRIMARY_COMMODITY"].eq("Bauxite")
grades.loc[baux_mask_in_grades, "Ore_Grade"] = pd.to_numeric(
    grades.loc[baux_mask_in_grades, "Ore_Grade"], errors="coerce"
) * 0.529

# Convert Ore_Grade from percent to decimal (e.g., 45 -> 0.45)
grades["Ore_Grade"] = pd.to_numeric(grades["Ore_Grade"], errors="coerce") / 100.0

# Drop helper column to avoid duplicate columns on merge
grades = grades[["PROP_ID", "Ore_Grade"]]

# --- Merge Ore_Grade into production (left join on PROP_ID) ---
prod = prod.merge(grades, on="PROP_ID", how="left")

# --- Fill missing Ore_Grade for specific commodities (fallbacks) ---
iron_ore_factors = {
    "China": 0.27, "Australia": 0.54, "Brazil": 0.64, "India": 0.61, "Russia": 0.60,
    "Ukraine": 0.69, "South Africa": 0.58, "Canada": 0.62, "United States": 0.57, "Sweden": 0.60
}
default_iron_ore_factor = 0.53

missing_grade = prod["Ore_Grade"].isna()

# Iron Ore: country-specific or default
mask_fe = missing_grade & (prod["PRIMARY_COMMODITY"] == "Iron Ore")
prod.loc[mask_fe, "Ore_Grade"] = prod.loc[mask_fe, "COUNTRY_NAME"].map(iron_ore_factors).fillna(default_iron_ore_factor)

# Manganese: 0.445
mask_mn = missing_grade & (prod["PRIMARY_COMMODITY"] == "Manganese")
prod.loc[mask_mn, "Ore_Grade"] = 0.445

# Bauxite: keep default fallback as-is (0.245) — only applies where grade was missing
mask_bx = missing_grade & (prod["PRIMARY_COMMODITY"] == "Bauxite")
prod.loc[mask_bx, "Ore_Grade"] = 0.245

# --- Place Ore_Grade column right after Reserves_Contained (if present) ---
if "Reserves_Contained" in prod.columns and "Ore_Grade" in prod.columns:
    cols = prod.columns.tolist()
    cols.remove("Ore_Grade")
    insert_at = cols.index("Reserves_Contained") + 1
    cols[insert_at:insert_at] = ["Ore_Grade"]
    prod = prod[cols]

# --- Save (overwrites) ---
prod.to_excel(prod_path, index=False)

# --- Optional: quick summary prints ---
filled_fe = mask_fe.sum()
filled_mn = mask_mn.sum()
filled_bx = mask_bx.sum()
merged_from_file = (~missing_grade & prod["PRIMARY_COMMODITY"].isin(["Iron Ore", "Bauxite", "Manganese"])).sum()

print("Merge complete. Applied 0.529 multiplier to bauxite grades from ore-grade file (before %→decimal).")
print(f" - From ore-grade file (targets): {merged_from_file}")
print(f" - Filled Iron Ore via country/default: {filled_fe}")
print(f" - Filled Manganese (0.445): {filled_mn}")
print(f" - Filled Bauxite default (0.245): {filled_bx}")
print(f"Saved to: {prod_path}")




Merge complete. Applied 0.529 multiplier to bauxite grades from ore-grade file (before %→decimal).
 - From ore-grade file (targets): 161
 - Filled Iron Ore via country/default: 306
 - Filled Manganese (0.445): 11
 - Filled Bauxite default (0.245): 28
Saved to: Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx


In [166]:
import pandas as pd
import re

# Paths
in_path = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
out_path = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"

# Load
df = pd.read_excel(in_path, dtype={"PROP_ID": str})

# Ensure Ore_Grade exists and is numeric (convert % -> decimal if needed)
if "Ore_Grade" not in df.columns:
    raise KeyError("Column 'Ore_Grade' not found in the input file.")
df["Ore_Grade"] = pd.to_numeric(df["Ore_Grade"], errors="coerce")
df["Ore_Grade"] = df["Ore_Grade"].where(df["Ore_Grade"] <= 1, df["Ore_Grade"] / 100.0)

# Identify year columns 1980..2023
year_cols = [c for c in df.columns
             if re.fullmatch(r"\d{4}", str(c)) and 1980 <= int(str(c)) <= 2023]

# Make sure year columns are numeric
for c in year_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Target mines: Iron Ore and Manganese
mask = df["PRIMARY_COMMODITY"].isin(["Iron Ore", "Manganese"])

# Multiply year columns by Ore_Grade (rows with missing grade remain unchanged)
before_nonnull = df.loc[mask, year_cols].notna().sum().sum()
df.loc[mask, year_cols] = df.loc[mask, year_cols].mul(df.loc[mask, "Ore_Grade"].fillna(1.0), axis=0)
after_nonnull = df.loc[mask, year_cols].notna().sum().sum()

# Save
df.to_excel(out_path, index=False)

# Simple summary
targets = mask.sum()
used_grades = df.loc[mask, "Ore_Grade"].notna().sum()
print("Applied Ore_Grade to year columns for Iron Ore and Manganese mines.")
print(f"Targets (rows): {targets}")
print(f"Rows with Ore_Grade applied: {used_grades}")
print(f"Saved to: {out_path}")


Applied Ore_Grade to year columns for Iron Ore and Manganese mines.
Targets (rows): 448
Rows with Ore_Grade applied: 448
Saved to: Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx


In [167]:
import pandas as pd

def merge_startup_years():
    """
    Merges START_UP_YEAR from the metadata file into START_UP_YR in the production file.
    Only fills empty cells in the production file with values from the metadata file.
    Merge is based on the shared PROP_ID column.
    """
    # Define file paths
    metadata_file = "Data Input/Meta Data/Lit_Missing_Start_Yr.xlsx"
    production_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
    
    print(f"Reading metadata file: {metadata_file}")
    
    try:
        # Read the metadata file
        metadata_df = pd.read_excel(metadata_file)
        print(f"Metadata file loaded: {len(metadata_df)} rows")
        
        # Check required columns in metadata file
        if 'PROP_ID' not in metadata_df.columns:
            print("Error: PROP_ID column not found in metadata file")
            return
        if 'START_UP_YEAR' not in metadata_df.columns:
            print("Error: START_UP_YEAR column not found in metadata file")
            return
        
        print(f"Reading production file: {production_file}")
        
        # Read the production file
        production_df = pd.read_excel(production_file)
        print(f"Production file loaded: {len(production_df)} rows")
        
        # Check required columns in production file
        if 'PROP_ID' not in production_df.columns:
            print("Error: PROP_ID column not found in production file")
            return
        if 'START_UP_YR' not in production_df.columns:
            print("Error: START_UP_YR column not found in production file")
            return
        
        # Count empty cells in production file before merge
        empty_startup_before = production_df['START_UP_YR'].isna().sum()
        print(f"Empty START_UP_YR cells in production file: {empty_startup_before}")
        
        # Create lookup dictionary from metadata file
        # Only include non-null values
        metadata_lookup = metadata_df[metadata_df['START_UP_YEAR'].notna()].set_index('PROP_ID')['START_UP_YEAR'].to_dict()
        print(f"Valid START_UP_YEAR values in metadata: {len(metadata_lookup)}")
        
        # Track changes
        cells_filled = 0
        
        # Fill empty cells in production file
        for idx, row in production_df.iterrows():
            prop_id = row['PROP_ID']
            current_startup = row['START_UP_YR']
            
            # Only fill if current cell is empty and we have data in metadata
            if pd.isna(current_startup) and prop_id in metadata_lookup:
                new_startup = metadata_lookup[prop_id]
                production_df.at[idx, 'START_UP_YR'] = new_startup
                cells_filled += 1
                print(f"PROP_ID {prop_id}: Filled empty START_UP_YR with {new_startup}")
        
        # Count empty cells after merge
        empty_startup_after = production_df['START_UP_YR'].isna().sum()
        
        # Delete rows that still have empty START_UP_YR after merging
        rows_before_deletion = len(production_df)
        production_df = production_df[production_df['START_UP_YR'].notna()]
        rows_after_deletion = len(production_df)
        rows_deleted = rows_before_deletion - rows_after_deletion
        
        print(f"Deleted {rows_deleted} rows with empty START_UP_YR after merging")
        
        # Save the updated production file
        production_df.to_excel(production_file, index=False)
        
        # Report results
        print(f"\n=== MERGE RESULTS ===")
        print(f"Empty START_UP_YR cells before merge: {empty_startup_before}")
        print(f"Empty START_UP_YR cells after merge: {empty_startup_after}")
        print(f"Cells filled: {cells_filled}")
        print(f"Rows deleted due to empty START_UP_YR: {rows_deleted}")
        print(f"Final rows remaining: {rows_after_deletion}")
        print(f"Total rows removed from dataset: {rows_before_deletion - rows_after_deletion}")
        
        # Show matching statistics
        metadata_prop_ids = set(metadata_df['PROP_ID'].dropna())
        production_prop_ids = set(production_df['PROP_ID'].dropna())
        matching_prop_ids = metadata_prop_ids.intersection(production_prop_ids)
        
        print(f"\nMatching statistics:")
        print(f"PROP_IDs in metadata file: {len(metadata_prop_ids)}")
        print(f"PROP_IDs in production file: {len(production_prop_ids)}")
        print(f"Matching PROP_IDs: {len(matching_prop_ids)}")
        
        print(f"\nUpdated production file saved: {production_file}")
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except Exception as e:
        print(f"Error during merge process: {e}")

if __name__ == "__main__":
    merge_startup_years()

Reading metadata file: Data Input/Meta Data/Lit_Missing_Start_Yr.xlsx
Metadata file loaded: 150 rows
Reading production file: Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx
Production file loaded: 1085 rows
Empty START_UP_YR cells in production file: 161
Valid START_UP_YEAR values in metadata: 150
PROP_ID 62580: Filled empty START_UP_YR with 1970
PROP_ID 55209: Filled empty START_UP_YR with 1940
PROP_ID 65193: Filled empty START_UP_YR with 1970
PROP_ID 68363: Filled empty START_UP_YR with 1985
PROP_ID 66249: Filled empty START_UP_YR with 1958
PROP_ID 68362: Filled empty START_UP_YR with 1983
PROP_ID 57501: Filled empty START_UP_YR with 1962
PROP_ID 68527: Filled empty START_UP_YR with 1987
PROP_ID 57368: Filled empty START_UP_YR with 1965
PROP_ID 69079: Filled empty START_UP_YR with 1948
PROP_ID 65191: Filled empty START_UP_YR with 1934
PROP_ID 64560: Filled empty START_UP_YR with 1965
PROP_ID 68360: Filled empty START_UP_YR with 1990
PROP_ID 57084: Filled empty S

In [168]:
import pandas as pd

def merge_closure_years():
    """
    Merges PROJ_CLOSURE_YR_Updated from the metadata file into PROJ_CLOSURE_YR 
    in the production file. Only merges valid years from the source dataset.
    Merge is based on the shared PROP_ID column.
    """
    # Define file paths
    metadata_file = "Data Input/Meta Data/Property_Closure_Analysis - Copy.xlsx"
    production_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
    
    print(f"Reading metadata file: {metadata_file}")
    
    try:
        # Read the metadata file
        metadata_df = pd.read_excel(metadata_file)
        print(f"Metadata file loaded: {len(metadata_df)} rows")
        
        # Check required columns in metadata file
        if 'PROP_ID' not in metadata_df.columns:
            print("Error: PROP_ID column not found in metadata file")
            return
        if 'PROJ_CLOSURE_YR_Updated' not in metadata_df.columns:
            print("Error: PROJ_CLOSURE_YR_Updated column not found in metadata file")
            return
        
        print(f"Reading production file: {production_file}")
        
        # Read the production file
        production_df = pd.read_excel(production_file)
        print(f"Production file loaded: {len(production_df)} rows")
        
        # Check required columns in production file
        if 'PROP_ID' not in production_df.columns:
            print("Error: PROP_ID column not found in production file")
            return
        if 'PROJ_CLOSURE_YR' not in production_df.columns:
            print("Error: PROJ_CLOSURE_YR column not found in production file")
            return
        
        # Convert closure year columns to numeric to validate years
        metadata_df['PROJ_CLOSURE_YR_Updated'] = pd.to_numeric(metadata_df['PROJ_CLOSURE_YR_Updated'], errors='coerce')
        production_df['PROJ_CLOSURE_YR'] = pd.to_numeric(production_df['PROJ_CLOSURE_YR'], errors='coerce')
        
        # Create lookup dictionary from metadata file - only include valid numerical values
        # Filter for non-null values that are valid numbers
        valid_closure_data = metadata_df[
            (metadata_df['PROJ_CLOSURE_YR_Updated'].notna())
        ]
        
        # Remove duplicates (keep first occurrence if there are multiple entries for same PROP_ID)
        valid_closure_data = valid_closure_data.drop_duplicates(subset=['PROP_ID'])
        
        closure_lookup = valid_closure_data.set_index('PROP_ID')['PROJ_CLOSURE_YR_Updated'].to_dict()
        print(f"Valid PROJ_CLOSURE_YR_Updated values in metadata: {len(closure_lookup)}")
        
        # Count existing closure data before merge
        existing_closure_before = production_df['PROJ_CLOSURE_YR'].notna().sum()
        print(f"Existing PROJ_CLOSURE_YR values in production file: {existing_closure_before}")
        
        # Track changes
        rows_updated = 0
        rows_added = 0
        
        # Update closure years in production file
        for idx, row in production_df.iterrows():
            prop_id = row['PROP_ID']
            current_closure = row['PROJ_CLOSURE_YR']
            
            # Check if we have updated closure data for this property
            if prop_id in closure_lookup:
                new_closure = closure_lookup[prop_id]
                
                if pd.isna(current_closure):
                    # Adding new closure year
                    production_df.at[idx, 'PROJ_CLOSURE_YR'] = new_closure
                    rows_added += 1
                    print(f"PROP_ID {prop_id}: Added closure year {int(new_closure)}")
                elif current_closure != new_closure:
                    # Updating existing closure year
                    production_df.at[idx, 'PROJ_CLOSURE_YR'] = new_closure
                    rows_updated += 1
                    print(f"PROP_ID {prop_id}: Updated closure year from {int(current_closure)} to {int(new_closure)}")
        
        # Count closure data after merge
        existing_closure_after = production_df['PROJ_CLOSURE_YR'].notna().sum()
        
        # Save the updated production file
        production_df.to_excel(production_file, index=False)
        
        # Report results
        print(f"\n=== MERGE RESULTS ===")
        print(f"PROJ_CLOSURE_YR values before merge: {existing_closure_before}")
        print(f"PROJ_CLOSURE_YR values after merge: {existing_closure_after}")
        print(f"Rows with new closure years added: {rows_added}")
        print(f"Rows with closure years updated: {rows_updated}")
        print(f"Total changes made: {rows_added + rows_updated}")
        print(f"Net increase in closure data: {existing_closure_after - existing_closure_before}")
        
        # Show matching statistics
        metadata_prop_ids = set(metadata_df['PROP_ID'].dropna())
        production_prop_ids = set(production_df['PROP_ID'].dropna())
        matching_prop_ids = metadata_prop_ids.intersection(production_prop_ids)
        
        print(f"\nMatching statistics:")
        print(f"PROP_IDs in metadata file: {len(metadata_prop_ids)}")
        print(f"PROP_IDs in production file: {len(production_prop_ids)}")
        print(f"Matching PROP_IDs: {len(matching_prop_ids)}")
        print(f"PROP_IDs with valid closure updates: {len(closure_lookup)}")
        
        print(f"\nUpdated production file saved: {production_file}")
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except Exception as e:
        print(f"Error during merge process: {e}")

if __name__ == "__main__":
    merge_closure_years()

Reading metadata file: Data Input/Meta Data/Property_Closure_Analysis - Copy.xlsx
Metadata file loaded: 62 rows
Reading production file: Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx
Production file loaded: 991 rows
Valid PROJ_CLOSURE_YR_Updated values in metadata: 26
Existing PROJ_CLOSURE_YR values in production file: 591
PROP_ID 29291: Updated closure year from 2009 to 2030
PROP_ID 27589: Updated closure year from 1997 to 2034
PROP_ID 28268: Updated closure year from 1987 to 2028
PROP_ID 31323: Updated closure year from 2022 to 2021
PROP_ID 28286: Updated closure year from 2022 to 2027
PROP_ID 27249: Updated closure year from 1999 to 2035
PROP_ID 27030: Updated closure year from 2020 to 2036
PROP_ID 30969: Updated closure year from 2016 to 2023
PROP_ID 27752: Updated closure year from 2015 to 2024
PROP_ID 33208: Updated closure year from 2021 to 2026
PROP_ID 31818: Updated closure year from 2012 to 2015
PROP_ID 31560: Updated closure year from 2019 to 2024
PROP

In [169]:
import pandas as pd

def correct_startup_years(file_path):
    """
    Corrects the START_UP_YR and PROJ_CLOSURE_YR for each mine based on actual production data.
    - If there's non-zero production before the recorded startup year, changes it to the year of first non-zero production.
    - If there's non-zero production after the projected closure year, changes it to the year of last non-zero production.
   
    Args:
        file_path (str): Path to the Excel file
    """
    print(f"Reading file: {file_path}")
   
    # Read the dataset
    df = pd.read_excel(file_path)
   
    # Check if required columns exist
    if 'START_UP_YR' not in df.columns:
        print("Error: START_UP_YR column not found")
        return
   
    if 'PROP_ID' not in df.columns:
        print("Error: PROP_ID column not found")
        return
    
    # Check if PROJ_CLOSURE_YR exists (optional for closure year correction)
    has_closure_data = 'PROJ_CLOSURE_YR' in df.columns
    if not has_closure_data:
        print("Warning: PROJ_CLOSURE_YR column not found - will only correct startup years")
   
    # Get year columns (1980-2023)
    year_columns = [str(year) for year in range(1980, 2024)]
    existing_year_columns = [col for col in year_columns if col in df.columns]
    existing_year_columns.sort(key=int)  # Ensure chronological order
   
    print(f"Found {len(existing_year_columns)} year columns")
   
    # Convert year columns to numeric
    for col in existing_year_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert closure year to numeric if it exists
    if has_closure_data:
        df['PROJ_CLOSURE_YR'] = pd.to_numeric(df['PROJ_CLOSURE_YR'], errors='coerce')
   
    # Track changes
    startup_changes_made = 0
    closure_changes_made = 0
   
    # Process each mine
    for idx, row in df.iterrows():
        startup_year = row['START_UP_YR']
        closure_year = row['PROJ_CLOSURE_YR'] if has_closure_data else None
       
        # Skip if no valid startup year
        if pd.isna(startup_year):
            continue
       
        startup_year = int(startup_year)
       
        # Find first year with non-zero production
        first_production_year = None
        for year_col in existing_year_columns:
            year = int(year_col)
            production = row[year_col]
           
            # Check if there's non-zero production
            if not pd.isna(production) and production > 0:
                first_production_year = year
                break
       
        # If first production is before recorded startup year, update it
        if first_production_year and first_production_year < startup_year:
            df.at[idx, 'START_UP_YR'] = first_production_year
            startup_changes_made += 1
            print(f"PROP_ID {row['PROP_ID']}: Changed startup year from {startup_year} to {first_production_year}")
        
        # Process closure year correction if closure data exists
        if has_closure_data and not pd.isna(closure_year):
            closure_year = int(closure_year)
            
            # Find last year with non-zero production after closure year
            last_production_after_closure = None
            for year_col in reversed(existing_year_columns):  # Start from latest year
                year = int(year_col)
                production = row[year_col]
                
                # Check if there's non-zero production after closure year
                if year > closure_year and not pd.isna(production) and production > 0:
                    last_production_after_closure = year
                    break  # We want the last (latest) year, so break on first match when going backwards
            
            # If production exists after closure year, update closure year
            if last_production_after_closure:
                df.at[idx, 'PROJ_CLOSURE_YR'] = last_production_after_closure
                closure_changes_made += 1
                print(f"PROP_ID {row['PROP_ID']}: Changed closure year from {closure_year} to {last_production_after_closure}")
   
    # Save the updated file
    df.to_excel(file_path, index=False)
   
    print(f"\nCompleted!")
    print(f"Made {startup_changes_made} changes to startup years")
    if has_closure_data:
        print(f"Made {closure_changes_made} changes to closure years")
    print(f"File saved: {file_path}")

if __name__ == "__main__":
    # Define file path
    file_path = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
   
    # Run the correction
    correct_startup_years(file_path)

Reading file: Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx
Found 44 year columns
PROP_ID 56954: Changed startup year from 2014 to 2011
PROP_ID 58083: Changed startup year from 2019 to 2010
PROP_ID 62576: Changed startup year from 2006 to 1984
PROP_ID 58076: Changed startup year from 2008 to 2002
PROP_ID 58546: Changed startup year from 2015 to 2000
PROP_ID 60813: Changed startup year from 2009 to 2008
PROP_ID 58079: Changed startup year from 2008 to 2004
PROP_ID 58534: Changed startup year from 2015 to 1996
PROP_ID 55265: Changed startup year from 2013 to 2011
PROP_ID 58535: Changed startup year from 2015 to 2002
PROP_ID 52676: Changed startup year from 2018 to 1980
PROP_ID 62582: Changed startup year from 2015 to 2000
PROP_ID 58077: Changed startup year from 2008 to 2002
PROP_ID 26483: Changed startup year from 2010 to 1990
PROP_ID 30526: Changed startup year from 2018 to 2008
PROP_ID 26908: Changed startup year from 2008 to 1986
PROP_ID 28272: Changed startup 

In [170]:
import pandas as pd

def fix_zero_startup_years(file_path):
    """
    Modifies the START_UP_YR for mines that have zero production in their startup year.
    If a mine's startup year shows 0 production, changes it to the first year with non-zero production.
    
    Args:
        file_path (str): Path to the Excel file
    """
    print(f"Reading file: {file_path}")
    
    # Read the dataset
    df = pd.read_excel(file_path)
    
    # Check if required columns exist
    if 'START_UP_YR' not in df.columns:
        print("Error: START_UP_YR column not found")
        return
    
    if 'PROP_ID' not in df.columns:
        print("Error: PROP_ID column not found")
        return
    
    # Get year columns (1980-2023)
    year_columns = [str(year) for year in range(1980, 2024)]
    existing_year_columns = [col for col in year_columns if col in df.columns]
    existing_year_columns.sort(key=int)  # Ensure chronological order
    
    print(f"Found {len(existing_year_columns)} year columns")
    
    # Convert year columns to numeric
    for col in existing_year_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert START_UP_YR to numeric
    df['START_UP_YR'] = pd.to_numeric(df['START_UP_YR'], errors='coerce')
    
    # Track changes
    changes_made = 0
    changed_prop_ids = []
    
    # Process each mine
    for idx, row in df.iterrows():
        startup_year = row['START_UP_YR']
        prop_id = row['PROP_ID']
        
        # Skip if no valid startup year
        if pd.isna(startup_year):
            continue
        
        startup_year = int(startup_year)
        startup_year_str = str(startup_year)
        
        # Check if startup year column exists in our data
        if startup_year_str not in existing_year_columns:
            print(f"Warning: Startup year {startup_year} for PROP_ID {prop_id} is outside data range")
            continue
        
        # Get production value for the startup year
        startup_production = row[startup_year_str]
        
        # Check if production in startup year is exactly 0 (excluding NaN/empty cells)
        if not pd.isna(startup_production) and startup_production == 0:
            
            # Find first year with non-zero production
            first_production_year = None
            for year_col in existing_year_columns:
                year = int(year_col)
                production = row[year_col]
                
                # Check if there's non-zero production
                if not pd.isna(production) and production > 0:
                    first_production_year = year
                    break
            
            # If we found a year with non-zero production, update startup year
            if first_production_year:
                df.at[idx, 'START_UP_YR'] = first_production_year
                changes_made += 1
                changed_prop_ids.append(prop_id)
                print(f"PROP_ID {prop_id}: Changed startup year from {startup_year} to {first_production_year} (startup year had zero production)")
            else:
                print(f"Warning: PROP_ID {prop_id} has zero production in startup year {startup_year} but no non-zero production found in any year")
    
    # Save the updated file
    df.to_excel(file_path, index=False)
    
    # Report results
    print(f"\n=== RESULTS ===")
    print(f"Total mines processed: {len(df)}")
    print(f"Mines with startup year corrections: {changes_made}")
    
    if changed_prop_ids:
        print(f"\nList of PROP_IDs that were changed:")
        for i, prop_id in enumerate(changed_prop_ids, 1):
            print(f"{i:3d}. {prop_id}")
    else:
        print("\nNo mines required startup year corrections.")
    
    print(f"\nFile saved: {file_path}")

if __name__ == "__main__":
    # Define file path
    file_path = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
    
    # Run the correction
    fix_zero_startup_years(file_path)

Reading file: Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx
Found 44 year columns
PROP_ID 80820: Changed startup year from 2015 to 2016 (startup year had zero production)
PROP_ID 81246: Changed startup year from 2015 to 2016 (startup year had zero production)
PROP_ID 81644: Changed startup year from 2015 to 2017 (startup year had zero production)
PROP_ID 33414: Changed startup year from 2014 to 2015 (startup year had zero production)
PROP_ID 29744: Changed startup year from 2003 to 2016 (startup year had zero production)
PROP_ID 37425: Changed startup year from 2004 to 2009 (startup year had zero production)
PROP_ID 30531: Changed startup year from 2006 to 2007 (startup year had zero production)
PROP_ID 80823: Changed startup year from 2015 to 2018 (startup year had zero production)
PROP_ID 80801: Changed startup year from 2011 to 2016 (startup year had zero production)
PROP_ID 36374: Changed startup year from 2015 to 2016 (startup year had zero production)
PROP_

Mean Imputation

In [171]:
import pandas as pd
import numpy as np
import os

# Define the mapping of countries to regions
COUNTRY_TO_REGION = {
    # Africa
    'Ghana': 'Africa',
    'Guinea': 'Africa',
    'Sierra Leone': 'Africa',
    'South Africa': 'Africa',
    'Mozambique': 'Africa',
    'Zimbabwe': 'Africa',
    'Botswana': 'Africa',
    'Tanzania': 'Africa',
    'Morocco': 'Africa',
    'Dem. Rep. Congo': 'Africa',
    'Zambia': 'Africa',
    'Namibia': 'Africa',
    'Mauritania': 'Africa',
    'Madagascar': 'Africa',
    'Liberia': 'Africa',
    'Algeria': 'Africa',
    'Gabon': 'Africa',
    'Rwanda': 'Africa',
    'Eritrea': 'Africa',
    'Burkina Faso': 'Africa',
   
    # Asia
    'Saudi Arabia': 'Middle East',
    'India': 'Asia',
    'China': 'Asia',
    'Kazakhstan': 'Asia',
    'Russia': 'Eurasia',
    'Indonesia': 'Asia',
    'Mongolia': 'Asia',
    'Uzbekistan': 'Asia',
    'Bangladesh': 'Asia',
    'Philippines': 'Asia',
    'Türkiye': 'Eurasia',
    'Pakistan': 'Asia',
    'Thailand': 'Asia',
    'Myanmar': 'Asia',
    'Laos': 'Asia',
    'Iran': 'Middle East',
    'Armenia': 'Eurasia',
    'Sri Lanka': 'Asia',
    'Malaysia': 'Asia',
    'Tajikistan': 'Asia',
    'South Korea': 'Asia',
    'Vietnam': 'Asia',
   
    # Europe
    'Greece': 'Europe',
    'Hungary': 'Europe',
    'Montenegro': 'Europe',
    'Ukraine': 'Europe',
    'Norway': 'Europe',
    'Czechia': 'Europe',
    'Poland': 'Europe',
    'Slovenia': 'Europe',
    'Spain': 'Europe',
    'Serbia': 'Europe',
    'Germany': 'Europe',
    'United Kingdom': 'Europe',
    'Bulgaria': 'Europe',
    'Sweden': 'Europe',
    'Portugal': 'Europe',
    'Romania': 'Europe',
    'North Macedonia': 'Europe',
    'Albania': 'Europe',
    'Georgia': 'Eurasia',
    'Cyprus': 'Europe',
    'Austria': 'Europe',
    'Bosnia & Herzegovina': 'Europe',
    'Finland': 'Europe',
    'Ireland': 'Europe',
   
    # North America
    'USA': 'North America',
    'Canada': 'North America',
    'Mexico': 'North America',
    'Dominican Republic': 'Caribbean',
    'Jamaica': 'Caribbean',
    'Panama': 'Central America',
    'Guatemala': 'Central America',
    'Cuba': 'Caribbean',
    'Honduras': 'Central America',
   
    # South America
    'Guyana': 'South America',
    'Brazil': 'South America',
    'Venezuela': 'South America',
    'Colombia': 'South America',
    'Argentina': 'South America',
    'Chile': 'South America',
    'Peru': 'South America',
    'Ecuador': 'South America',
    'Bolivia': 'South America',
   
    # Oceania
    'Australia': 'Oceania',
    'New Zealand': 'Oceania',
    'Papua New Guinea': 'Oceania',
    'New Caledonia': 'Oceania'
}

def main():
    # Path to the input and output files
    input_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
    output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_imputed.xlsx"
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Read the Excel file
    print(f"Reading data from {input_file}...")
    df = pd.read_excel(input_file)
    
    # Store initial row count
    initial_rows = len(df)
    
    # Add a REGION column based on COUNTRY_NAME
    print("Adding REGION column...")
    df['REGION'] = df['COUNTRY_NAME'].map(COUNTRY_TO_REGION)
    
    # Calculate mine lifetimes for mines with both START_UP_YR and PROJ_CLOSURE_YR
    print("Calculating mine lifetimes...")
    df['MINE_LIFETIME'] = None
    mask = (df['START_UP_YR'].notna()) & (df['PROJ_CLOSURE_YR'].notna())
    df.loc[mask, 'MINE_LIFETIME'] = df.loc[mask, 'PROJ_CLOSURE_YR'] - df.loc[mask, 'START_UP_YR']
    
    # Create a copy of the original PROJ_CLOSURE_YR column to compare later
    df['ORIGINAL_PROJ_CLOSURE_YR'] = df['PROJ_CLOSURE_YR']
    
    # Count missing values before imputation
    missing_before = df['PROJ_CLOSURE_YR'].isna().sum()
    print(f"Missing PROJ_CLOSURE_YR values before imputation: {missing_before}")
    
    # Filter rows with valid mine lifetimes for calculating medians
    lifetime_df = df[(df['MINE_LIFETIME'].notna()) & (df['MINE_LIFETIME'] > 0)]
    
    print(f"Number of mines with valid lifetimes (for median calculation): {len(lifetime_df)}")
    
    # Calculate median lifetimes and counts for each level of aggregation
    # Level 1: PRIMARY_COMMODITY + COUNTRY_NAME + MINE_TYPE1
    level1_medians = lifetime_df.groupby(['PRIMARY_COMMODITY', 'COUNTRY_NAME', 'MINE_TYPE1'])['MINE_LIFETIME'].agg(['median', 'count'])
    # Level 2: PRIMARY_COMMODITY + REGION + MINE_TYPE1
    level2_medians = lifetime_df.groupby(['PRIMARY_COMMODITY', 'REGION', 'MINE_TYPE1'])['MINE_LIFETIME'].agg(['median', 'count'])
    # Level 3: PRIMARY_COMMODITY + MINE_TYPE1 (global)
    level3_medians = lifetime_df.groupby(['PRIMARY_COMMODITY', 'MINE_TYPE1'])['MINE_LIFETIME'].agg(['median', 'count'])
    
    # Print summary of available combinations
    print(f"\nLevel 1 (Commodity+Country+MineType) combinations: {len(level1_medians)}")
    print(f"Level 2 (Commodity+Region+MineType) combinations: {len(level2_medians)}")
    print(f"Level 3 (Commodity+MineType) combinations: {len(level3_medians)}")
    
    # Apply hierarchical imputation with the new rules
    print("\nApplying hierarchical imputation based on median lifetimes...")
    print("Using criteria: Check next level if lifetime < 30 or > 50 years, or if sample count < 5")
    
    # Create a column to track imputation level
    if 'IMPUTATION_LEVEL' not in df.columns:
        df['IMPUTATION_LEVEL'] = None
    
    # Create counters for each imputation case
    level1_count = 0
    level2_count = 0
    level3_count = 0
    skipped_level1_low_lifetime = 0
    skipped_level1_high_lifetime = 0
    skipped_level1_small_sample = 0
    skipped_level2_low_lifetime = 0
    skipped_level2_high_lifetime = 0
    skipped_level2_small_sample = 0
    skipped_level3_small_sample = 0 

    # Track the mines that will be imputed to help with consistency check later
    imputed_mines = []
    
    # Loop through rows with missing PROJ_CLOSURE_YR but valid START_UP_YR
    for idx, row in df[(df['PROJ_CLOSURE_YR'].isna()) & (df['START_UP_YR'].notna())].iterrows():
        key1 = (row['PRIMARY_COMMODITY'], row['COUNTRY_NAME'], row['MINE_TYPE1'])
        key2 = (row['PRIMARY_COMMODITY'], row['REGION'], row['MINE_TYPE1']) if pd.notna(row['REGION']) else None
        key3 = (row['PRIMARY_COMMODITY'], row['MINE_TYPE1'])
        
        # Try level 1 first
        use_level1 = False
        if key1 in level1_medians.index:
            lifetime = level1_medians.loc[key1, 'median']
            count = level1_medians.loc[key1, 'count']
            
            # Check if we should use this level
            if count >= 3 and 30 <= lifetime <= 50:
                use_level1 = True
            else:
                if count < 3:
                    skipped_level1_small_sample += 1
                elif lifetime < 30:
                    skipped_level1_low_lifetime += 1
                elif lifetime > 50:
                    skipped_level1_high_lifetime += 1
        
        if use_level1:
            # Use level 1 for imputation
            lifetime = int(round(lifetime))
            df.at[idx, 'PROJ_CLOSURE_YR'] = row['START_UP_YR'] + lifetime
            df.at[idx, 'IMPUTATION_LEVEL'] = 'Commodity+Country+MineType'
            df.at[idx, 'MINE_LIFETIME'] = lifetime
            imputed_mines.append(idx)
            level1_count += 1
            continue
        
        # Try level 2 if level 1 doesn't meet criteria
        use_level2 = False
        if key2 is not None and key2 in level2_medians.index:
            lifetime = level2_medians.loc[key2, 'median']
            count = level2_medians.loc[key2, 'count']
            
            # Check if we should use this level
            if count >= 3 and 30 <= lifetime <= 50:
                use_level2 = True
            else:
                if count < 3:
                    skipped_level2_small_sample += 1
                elif lifetime < 30:
                    skipped_level2_low_lifetime += 1
                elif lifetime > 50:
                    skipped_level2_high_lifetime += 1
        
        if use_level2:
            # Use level 2 for imputation
            lifetime = int(round(lifetime))
            df.at[idx, 'PROJ_CLOSURE_YR'] = row['START_UP_YR'] + lifetime
            df.at[idx, 'IMPUTATION_LEVEL'] = 'Commodity+Region+MineType'
            df.at[idx, 'MINE_LIFETIME'] = lifetime
            imputed_mines.append(idx)
            level2_count += 1
            continue
        
        # Try level 3 if level 2 doesn't meet criteria
        use_level3 = False
        if key3 in level3_medians.index:
            lifetime = level3_medians.loc[key3, 'median']
            count = level3_medians.loc[key3, 'count']

            # Level 3: only require at least 3 samples; no lifetime guardrails
            if count >= 3:
                use_level3 = True
            else:
                skipped_level3_small_sample += 1

        if use_level3:
            lifetime = int(round(lifetime))
            df.at[idx, 'PROJ_CLOSURE_YR'] = row['START_UP_YR'] + lifetime
            df.at[idx, 'IMPUTATION_LEVEL'] = 'Commodity+Global+MineType'
            df.at[idx, 'MINE_LIFETIME'] = lifetime
            imputed_mines.append(idx)
            level3_count += 1
     
    # Ensure all PROJ_CLOSURE_YR values are integers (for both original and imputed values)
    mask = df['PROJ_CLOSURE_YR'].notna()
    df.loc[mask, 'PROJ_CLOSURE_YR'] = df.loc[mask, 'PROJ_CLOSURE_YR'].apply(lambda x: int(round(x)))
    
    # Count missing values after imputation
    missing_after = df['PROJ_CLOSURE_YR'].isna().sum()
    imputed_count = level1_count + level2_count + level3_count
    print(f"\nImputation results:")
    print(f"Total imputed values: {imputed_count}")
    print(f"  Level 1 (Commodity+Country+MineType): {level1_count}")
    print(f"  Level 2 (Commodity+Region+MineType): {level2_count}")
    print(f"  Level 3 (Commodity+MineType): {level3_count}")
    
    print(f"\nSkipped imputation at Level 1 (used lower level instead):")
    print(f"  Due to small sample size (< 5): {skipped_level1_small_sample}")
    print(f"  Due to low lifetime (< 30 years): {skipped_level1_low_lifetime}")
    print(f"  Due to high lifetime (> 50 years): {skipped_level1_high_lifetime}")
    
    print(f"\nSkipped imputation at Level 2 (used lower level instead):")
    print(f"  Due to small sample size (< 5): {skipped_level2_small_sample}")
    print(f"  Due to low lifetime (< 30 years): {skipped_level2_low_lifetime}")
    print(f"  Due to high lifetime (> 50 years): {skipped_level2_high_lifetime}")

    print(f"\nSkipped imputation at Level 3 (left missing):")
    print(f"  Due to small sample size (< 3): {skipped_level3_small_sample}")

    print(f"\nMissing PROJ_CLOSURE_YR values after imputation: {missing_after}")
    
    if missing_before > 0:
        print(f"Imputation coverage: {100 * (missing_before - missing_after) / missing_before:.2f}%")
    else:
        print("No missing values were present before imputation.")
    
    # Find the year columns in the dataset
    year_columns = [col for col in df.columns if str(col).isdigit() and 1980 <= int(col) <= 2023]
    
    if year_columns:
        print("\nChecking for inconsistencies where imputed closure year is earlier than the latest production year...")
        
        inconsistent_mines = []
        corrected_mines = []
        
        # For each imputed mine, find the latest year with production data
        for idx in imputed_mines:
            row = df.loc[idx]
            
            # Extract the production data for years
            production_data = row[year_columns]
            
            # Find the latest year with a non-zero, non-NaN production value
            latest_production_year = None
            for year in sorted(year_columns, reverse=True):
                if pd.notna(row[year]) and row[year] > 0:
                    latest_production_year = int(year)
                    break
            
            # If there is production data and the imputed closure year is earlier
            if latest_production_year is not None and row['PROJ_CLOSURE_YR'] < latest_production_year:
                inconsistent_mines.append({
                    'PROP_ID': row['PROP_ID'],
                    'PRIMARY_COMMODITY': row['PRIMARY_COMMODITY'],
                    'COUNTRY_NAME': row['COUNTRY_NAME'],
                    'MINE_TYPE1': row['MINE_TYPE1'],
                    'START_UP_YR': row['START_UP_YR'],
                    'ORIGINAL_IMPUTED_CLOSURE_YEAR': row['PROJ_CLOSURE_YR'],
                    'LATEST_PRODUCTION_YEAR': latest_production_year,
                    'MINE_LIFETIME': row['MINE_LIFETIME'],
                    'IMPUTATION_LEVEL': row['IMPUTATION_LEVEL']
                })
                
                # Correct the closure year to the latest production year
                df.at[idx, 'PROJ_CLOSURE_YR'] = latest_production_year
                df.at[idx, 'MINE_LIFETIME'] = latest_production_year - row['START_UP_YR']
                corrected_mines.append(idx)
        
        # Report inconsistencies and corrections
        if inconsistent_mines:
            print(f"\nFound {len(inconsistent_mines)} mines where the imputed closure year was earlier than the latest production year")
            print(f"Corrected all {len(corrected_mines)} mines by setting closure year to latest production year")
            
            # Group inconsistencies by imputation level
            inconsistent_df = pd.DataFrame(inconsistent_mines)
            inconsistent_by_level = inconsistent_df['IMPUTATION_LEVEL'].value_counts()
            print("\nCorrections by original imputation level:")
            for level, count in inconsistent_by_level.items():
                print(f"  {level}: {count} mines")
        else:
            print("No inconsistencies found. All imputed closure years are after the latest production year.")
    else:
        print("\nCould not find year columns to check for production consistency.")
    
    # Remove rows with missing closure years after imputation
    rows_before_removal = len(df)
    
    # Identify rows that will be dropped (missing closure years)
    dropped_rows = df[df['PROJ_CLOSURE_YR'].isna()]
    
    # Save information about dropped mines
    if len(dropped_rows) > 0:
        dropped_info = dropped_rows[['PROP_ID', 'PRIMARY_COMMODITY', 'MINE_TYPE1']].copy()
        print(f"\n=== MINES TO BE DROPPED (Missing Closure Years) ===")
        print(f"Total mines to be dropped: {len(dropped_info)}")
        print("\nDetailed list of dropped mines:")
        print("-" * 80)
        print(f"{'PROP_ID':<15} {'PRIMARY_COMMODITY':<25} {'MINE_TYPE1':<30}")
        print("-" * 80)
        
        for idx, row in dropped_info.iterrows():
            prop_id = str(row['PROP_ID'])[:14]  # Truncate if too long
            commodity = str(row['PRIMARY_COMMODITY'])[:24]  # Truncate if too long
            mine_type = str(row['MINE_TYPE1'])[:29]  # Truncate if too long
            print(f"{prop_id:<15} {commodity:<25} {mine_type:<30}")
        
        # Summary by commodity and mine type
        print(f"\n{'='*80}")
        print("SUMMARY OF DROPPED MINES BY COMMODITY:")
        commodity_summary = dropped_info['PRIMARY_COMMODITY'].value_counts()
        for commodity, count in commodity_summary.items():
            print(f"  {commodity}: {count} mines")
        
        print(f"\nSUMMARY OF DROPPED MINES BY MINE TYPE:")
        mine_type_summary = dropped_info['MINE_TYPE1'].value_counts()
        for mine_type, count in mine_type_summary.items():
            print(f"  {mine_type}: {count} mines")
    
    # Remove the rows with missing closure years
    df = df[df['PROJ_CLOSURE_YR'].notna()]
    rows_after_removal = len(df)
    rows_removed = rows_before_removal - rows_after_removal
    
    print(f"\n=== FINAL CLEANUP ===")
    print(f"Removed {rows_removed} rows with missing closure years after imputation")
    print(f"Final dataset: {rows_after_removal} rows (started with {initial_rows} rows)")
    print(f"Total rows removed: {initial_rows - rows_after_removal}")
    
    # Save the imputed dataframe to Excel
    print(f"\nSaving cleaned and imputed data to {output_file}...")
    df.to_excel(output_file, index=False)
    print("Done!")

if __name__ == "__main__":
    main()

Reading data from Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx...
Adding REGION column...
Calculating mine lifetimes...
Missing PROJ_CLOSURE_YR values before imputation: 400
Number of mines with valid lifetimes (for median calculation): 591

Level 1 (Commodity+Country+MineType) combinations: 136
Level 2 (Commodity+Region+MineType) combinations: 68
Level 3 (Commodity+MineType) combinations: 14

Applying hierarchical imputation based on median lifetimes...
Using criteria: Check next level if lifetime < 30 or > 50 years, or if sample count < 5

Imputation results:
Total imputed values: 398
  Level 1 (Commodity+Country+MineType): 90
  Level 2 (Commodity+Region+MineType): 203
  Level 3 (Commodity+MineType): 105

Skipped imputation at Level 1 (used lower level instead):
  Due to small sample size (< 5): 36
  Due to low lifetime (< 30 years): 157
  Due to high lifetime (> 50 years): 38

Skipped imputation at Level 2 (used lower level instead):
  Due to small sample siz

In [172]:
import pandas as pd
import os

def merge_closure_years_final():
    """
    Merges PROJ_CLOSURE_YR from the imputed dataset into the original dataset
    based on PROP_ID, removes rows without closure years, and saves as final dataset.
    """
    # Define file paths
    imputed_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_imputed.xlsx"
    original_file = "Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx"
    output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    
    print(f"Reading imputed dataset: {imputed_file}")
    
    try:
        # Read the imputed dataset (source of closure years)
        imputed_df = pd.read_excel(imputed_file)
        print(f"Imputed dataset loaded: {len(imputed_df)} rows")
        
        # Check required columns in imputed file
        if 'PROP_ID' not in imputed_df.columns:
            print("Error: PROP_ID column not found in imputed file")
            return
        if 'PROJ_CLOSURE_YR' not in imputed_df.columns:
            print("Error: PROJ_CLOSURE_YR column not found in imputed file")
            return
        
        print(f"Reading original dataset: {original_file}")
        
        # Read the original dataset (target)
        original_df = pd.read_excel(original_file)
        print(f"Original dataset loaded: {len(original_df)} rows")
        
        # Check required columns in original file
        if 'PROP_ID' not in original_df.columns:
            print("Error: PROP_ID column not found in original file")
            return
        if 'PROJ_CLOSURE_YR' not in original_df.columns:
            print("Error: PROJ_CLOSURE_YR column not found in original file")
            return
        
        # Count closure years before merge
        closure_before = original_df['PROJ_CLOSURE_YR'].notna().sum()
        print(f"PROJ_CLOSURE_YR values in original dataset: {closure_before}")
        
        # Create lookup dictionary from imputed dataset
        # Only include non-null closure years
        closure_lookup = imputed_df[imputed_df['PROJ_CLOSURE_YR'].notna()].set_index('PROP_ID')['PROJ_CLOSURE_YR'].to_dict()
        print(f"Valid PROJ_CLOSURE_YR values in imputed dataset: {len(closure_lookup)}")
        
        # Track changes
        rows_updated = 0
        rows_added = 0
        
        # Update closure years in original dataset
        for idx, row in original_df.iterrows():
            prop_id = row['PROP_ID']
            current_closure = row['PROJ_CLOSURE_YR']
            
            # Check if we have closure data for this property
            if prop_id in closure_lookup:
                new_closure = closure_lookup[prop_id]
                
                if pd.isna(current_closure):
                    # Adding new closure year
                    original_df.at[idx, 'PROJ_CLOSURE_YR'] = new_closure
                    rows_added += 1
                elif current_closure != new_closure:
                    # Updating existing closure year
                    original_df.at[idx, 'PROJ_CLOSURE_YR'] = new_closure
                    rows_updated += 1
        
        # Count closure years after merge
        closure_after = original_df['PROJ_CLOSURE_YR'].notna().sum()
        
        print(f"\n=== MERGE RESULTS ===")
        print(f"PROJ_CLOSURE_YR values before merge: {closure_before}")
        print(f"PROJ_CLOSURE_YR values after merge: {closure_after}")
        print(f"Rows with new closure years added: {rows_added}")
        print(f"Rows with closure years updated: {rows_updated}")
        print(f"Total changes made: {rows_added + rows_updated}")
        
        # Identify rows that will be dropped (missing closure years)
        rows_before_removal = len(original_df)
        dropped_rows = original_df[original_df['PROJ_CLOSURE_YR'].isna()]
        
        # Save information about dropped mines
        if len(dropped_rows) > 0:
            dropped_info = dropped_rows[['PROP_ID', 'PRIMARY_COMMODITY', 'MINE_TYPE1']].copy()
            print(f"\n=== MINES TO BE DROPPED (Missing Closure Years) ===")
            print(f"Total mines to be dropped: {len(dropped_info)}")
            print("\nDetailed list of dropped mines:")
            print("-" * 80)
            print(f"{'PROP_ID':<15} {'PRIMARY_COMMODITY':<25} {'MINE_TYPE1':<30}")
            print("-" * 80)
            
            for idx, row in dropped_info.iterrows():
                prop_id = str(row['PROP_ID'])[:14]  # Truncate if too long
                commodity = str(row['PRIMARY_COMMODITY'])[:24]  # Truncate if too long
                mine_type = str(row['MINE_TYPE1'])[:29]  # Truncate if too long
                print(f"{prop_id:<15} {commodity:<25} {mine_type:<30}")
            
            # Summary by commodity and mine type
            print(f"\n{'='*80}")
            print("SUMMARY OF DROPPED MINES BY COMMODITY:")
            commodity_summary = dropped_info['PRIMARY_COMMODITY'].value_counts()
            for commodity, count in commodity_summary.items():
                print(f"  {commodity}: {count} mines")
            
            print(f"\nSUMMARY OF DROPPED MINES BY MINE TYPE:")
            mine_type_summary = dropped_info['MINE_TYPE1'].value_counts()
            for mine_type, count in mine_type_summary.items():
                print(f"  {mine_type}: {count} mines")
        
        # Remove rows with missing closure years
        final_df = original_df[original_df['PROJ_CLOSURE_YR'].notna()]
        rows_after_removal = len(final_df)
        rows_removed = rows_before_removal - rows_after_removal
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        # Save the final dataset
        final_df.to_excel(output_file, index=False)
        
        # Final reporting
        print(f"\n=== FINAL DATASET SUMMARY ===")
        print(f"Original dataset rows: {rows_before_removal}")
        print(f"Rows removed due to missing closure years: {rows_removed}")
        print(f"Final dataset rows: {rows_after_removal}")
        print(f"Data retention rate: {rows_after_removal/rows_before_removal*100:.1f}%")
        
        # Show matching statistics
        imputed_prop_ids = set(imputed_df['PROP_ID'].dropna())
        original_prop_ids = set(original_df['PROP_ID'].dropna())
        matching_prop_ids = imputed_prop_ids.intersection(original_prop_ids)
        
        print(f"\nMatching statistics:")
        print(f"PROP_IDs in imputed dataset: {len(imputed_prop_ids)}")
        print(f"PROP_IDs in original dataset: {len(original_prop_ids)}")
        print(f"Matching PROP_IDs: {len(matching_prop_ids)}")
        print(f"PROP_IDs with valid closure data transferred: {len(closure_lookup)}")
        
        print(f"\nFinal dataset saved: {output_file}")
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except Exception as e:
        print(f"Error during merge process: {e}")

if __name__ == "__main__":
    merge_closure_years_final()

Reading imputed dataset: Data Output/Commodity Production/Commodity_Production-1980_2023_imputed.xlsx
Imputed dataset loaded: 989 rows
Reading original dataset: Data Output/Commodity Production/Commodity_Production-1980_2023.xlsx
Original dataset loaded: 991 rows
PROJ_CLOSURE_YR values in original dataset: 591
Valid PROJ_CLOSURE_YR values in imputed dataset: 989

=== MERGE RESULTS ===
PROJ_CLOSURE_YR values before merge: 591
PROJ_CLOSURE_YR values after merge: 989
Rows with new closure years added: 398
Rows with closure years updated: 0
Total changes made: 398

=== MINES TO BE DROPPED (Missing Closure Years) ===
Total mines to be dropped: 2

Detailed list of dropped mines:
--------------------------------------------------------------------------------
PROP_ID         PRIMARY_COMMODITY         MINE_TYPE1                    
--------------------------------------------------------------------------------
70310           Lithium                   Underground                   
74801     

Preperation for CumProd
1) If there are greater than or equal to 3 consecutive zeros after the last non-zero production value and the projected closure year is past 2023 then drop the mine
2) If there are greater than or equal to 1 consecutive zeros after the last non-zero produciton value and the projected closure year is less than or equal to 2023, then change the projected closure year to the last year of non-zero production
3) If there are greater than or equal to 4 consectuvie zeros and not a total of greater than or equal to 5 non-zero production values after the consecutive zeros then drop the mine. 
4) If there are greater than or equal to 5 non-zero prroduction values after the greater than or equal to 3 consecutive zeros then flag the mine and print its PROP_ID.
5) Flag mines that are flagged in criteria 4 that have multiple greater than or equal to 3 consecutive zeroes

For the flagged mines
1) for the flagged mines of criteria 4, if the mine has greater than or equal to 1 consecutive zeros then change the start up year to the year of the first non-zero production. 
2) for the flagged mines of criteria 4, if the mine has greater than or equal to 5 consecutive zeros after the start up year then change the start up year to the year of the first non-zero production after the consecutvie zeros --> do this only for mines whose projected closure is greater than 2023, for the ones less than or equal to 2023, drop the mine. do this only after running the first criteria. 
2) leave the flagged mines of criteria 5 the same

*for the flagged mines of criteria 5, I will edit it manually through visual inspection

In [173]:
import pandas as pd
import numpy as np

def edit_production_dataset():
    """
    Edits the production dataset based on consecutive zero patterns and closure years.
    
    Criteria:
    1) ≥2 consecutive zeros after last production AND closure > 2023 → Drop mine
    2) ≥1 consecutive zeros after last production AND closure ≤ 2023 → Change closure to last production year
    3) ≥4 consecutive zeros AND not ≥5 non-zero values after zeros → Drop mine
    4) ≥5 non-zero values after ≥3 consecutive zeros → Flag mine
    5) Flag mines that are flagged in criteria 4 that have multiple ≥3 consecutive zero stretches
    """
    # Define file path
    input_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    
    print(f"Reading dataset: {input_file}")
    
    try:
        # Read the dataset
        df = pd.read_excel(input_file)
        print(f"Dataset loaded: {len(df)} rows")
        
        # Check required columns
        if 'PROP_ID' not in df.columns:
            print("Error: PROP_ID column not found")
            return
        if 'PROJ_CLOSURE_YR' not in df.columns:
            print("Error: PROJ_CLOSURE_YR column not found")
            return
        
        # Get year columns (1980-2023)
        year_columns = [str(year) for year in range(1980, 2024)]
        existing_year_columns = [col for col in year_columns if col in df.columns]
        existing_year_columns.sort(key=int)  # Ensure chronological order
        
        print(f"Found {len(existing_year_columns)} year columns")
        
        # Convert year columns to numeric
        for col in existing_year_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Convert closure year to numeric
        df['PROJ_CLOSURE_YR'] = pd.to_numeric(df['PROJ_CLOSURE_YR'], errors='coerce')
        
        # Initialize tracking variables
        initial_rows = len(df)
        mines_to_drop = []
        closure_changes = []
        flagged_mines = []
        multi_gap_flagged_mines = []
        
        # Process each mine
        for idx, row in df.iterrows():
            prop_id = row['PROP_ID']
            closure_year = row['PROJ_CLOSURE_YR']
            
            # Get production values for all years, keeping NaN as NaN
            production_values = []
            for year_col in existing_year_columns:
                value = row[year_col]
                production_values.append(value)  # Keep NaN as NaN, don't convert to 0
            
            # Find last non-zero production year (value > 0, not NaN)
            last_nonzero_idx = None
            for i in range(len(production_values) - 1, -1, -1):
                value = production_values[i]
                if not pd.isna(value) and value > 0:
                    last_nonzero_idx = i
                    break
            
            if last_nonzero_idx is None:
                # No non-zero production found, skip this mine
                continue
            
            last_nonzero_year = int(existing_year_columns[last_nonzero_idx])
            
            # Count consecutive ACTUAL zeros (not NaN) after last non-zero production
            consecutive_zeros_after = 0
            for i in range(last_nonzero_idx + 1, len(production_values)):
                value = production_values[i]
                if not pd.isna(value) and value == 0:  # Only count actual zeros
                    consecutive_zeros_after += 1
                elif not pd.isna(value) and value > 0:  # Stop if we hit non-zero production
                    break
                # If it's NaN, we skip it (don't count as zero, don't break the streak)
            
            # Check criteria 1 and 2 based on consecutive zeros after last production
            if pd.notna(closure_year):
                if closure_year > 2023 and consecutive_zeros_after >= 2:
                    # Criteria 1: ≥2 consecutive zeros after last production AND closure > 2023 → Drop mine
                    mines_to_drop.append({
                        'idx': idx,
                        'prop_id': prop_id,
                        'reason': 'Criteria 1: ≥2 zeros after last production, closure > 2023',
                        'consecutive_zeros': consecutive_zeros_after,
                        'last_production_year': last_nonzero_year,
                        'closure_year': closure_year
                    })
                    continue
                elif closure_year <= 2023 and consecutive_zeros_after >= 1:
                    # Criteria 2: ≥1 consecutive zeros after last production AND closure ≤ 2023 → Change closure to last production year
                    if closure_year != last_nonzero_year:
                        closure_changes.append({
                            'prop_id': prop_id,
                            'old_closure': closure_year,
                            'new_closure': last_nonzero_year,
                            'consecutive_zeros': consecutive_zeros_after
                        })
                        df.at[idx, 'PROJ_CLOSURE_YR'] = last_nonzero_year
                    continue
            
            # Find all stretches of ≥3 consecutive ACTUAL zeros in the entire timeline
            zero_stretches = []
            current_zeros = 0
            stretch_start = None
            
            for i in range(len(production_values)):
                value = production_values[i]
                if not pd.isna(value) and value == 0:  # Only count actual zeros
                    if current_zeros == 0:
                        stretch_start = i
                    current_zeros += 1
                elif not pd.isna(value) and value > 0:  # Hit non-zero production
                    if current_zeros >= 3 and stretch_start is not None:
                        # Record this zero stretch
                        zero_stretches.append({
                            'start': stretch_start,
                            'end': i - 1,
                            'length': current_zeros
                        })
                    current_zeros = 0
                    stretch_start = None
                # If NaN, we skip it (don't count as zero, don't reset counter)
            
            # Check for a zero stretch that extends to the end
            if current_zeros >= 3 and stretch_start is not None:
                zero_stretches.append({
                    'start': stretch_start,
                    'end': len(production_values) - 1,
                    'length': current_zeros
                })
            
            # Process zero stretches for criteria 3 and 4
            mine_flagged_criteria_4 = False
            
            for stretch in zero_stretches:
                if stretch['length'] >= 3:
                    # Count NON-ZERO values (> 0, not NaN) after this zero stretch
                    nonzero_after_stretch = 0
                    
                    for i in range(stretch['end'] + 1, len(production_values)):
                        value = production_values[i]
                        if not pd.isna(value) and value > 0:  # Only count actual non-zero values
                            nonzero_after_stretch += 1
                    
                    if nonzero_after_stretch >= 5:
                        # Criteria 4: Flag mine
                        if not mine_flagged_criteria_4:  # Only add once per mine
                            flagged_mines.append({
                                'prop_id': prop_id,
                                'zero_stretch_start_year': int(existing_year_columns[stretch['start']]),
                                'zero_stretch_end_year': int(existing_year_columns[stretch['end']]),
                                'zero_stretch_length': stretch['length'],
                                'nonzero_after_stretch': nonzero_after_stretch,
                                'total_zero_stretches': len([s for s in zero_stretches if s['length'] >= 3])
                            })
                            mine_flagged_criteria_4 = True
                    elif stretch['length'] >= 4:  # Only apply criteria 3 if stretch is ≥4 years
                        # Criteria 3: Drop mine (only for ≥4 consecutive zeros)
                        mines_to_drop.append({
                            'idx': idx,
                            'prop_id': prop_id,
                            'reason': f'Criteria 3: ≥4 consecutive zeros, only {nonzero_after_stretch} non-zero values after',
                            'zero_stretch_start_year': int(existing_year_columns[stretch['start']]),
                            'zero_stretch_end_year': int(existing_year_columns[stretch['end']]),
                            'zero_stretch_length': stretch['length'],
                            'nonzero_after_stretch': nonzero_after_stretch
                        })
                        break  # Don't process other stretches for this mine since we're dropping it
            
            # Criteria 5: Check if mine flagged in criteria 4 has multiple ≥3 zero stretches
            if mine_flagged_criteria_4:
                multiple_stretches_count = len([s for s in zero_stretches if s['length'] >= 3])
                if multiple_stretches_count > 1:
                    multi_gap_flagged_mines.append({
                        'prop_id': prop_id,
                        'total_zero_stretches': multiple_stretches_count,
                        'stretches_details': [{
                            'start_year': int(existing_year_columns[s['start']]),
                            'end_year': int(existing_year_columns[s['end']]),
                            'length': s['length']
                        } for s in zero_stretches if s['length'] >= 3]
                    })
        
        # Apply changes
        print(f"\n=== PROCESSING RESULTS ===")
        
        # Report closure year changes
        if closure_changes:
            print(f"\nCriteria 2 - Closure year changes: {len(closure_changes)}")
            for change in closure_changes:
                print(f"  PROP_ID {change['prop_id']}: Changed closure from {change['old_closure']} to {change['new_closure']} ({change['consecutive_zeros']} trailing zeros)")
        
        # Report flagged mines
        if flagged_mines:
            print(f"\nCriteria 4 - Flagged mines (≥5 non-zero values after ≥3 consecutive zeros): {len(flagged_mines)}")
            for flag in flagged_mines:
                print(f"  PROP_ID {flag['prop_id']}: Zero stretch {flag['zero_stretch_start_year']}-{flag['zero_stretch_end_year']} (length: {flag['zero_stretch_length']}), {flag['nonzero_after_stretch']} non-zero values after, {flag['total_zero_stretches']} total stretches")
        
        # Report multi-gap flagged mines
        if multi_gap_flagged_mines:
            print(f"\nCriteria 5 - Multi-gap flagged mines (flagged in criteria 4 with multiple ≥3 zero stretches): {len(multi_gap_flagged_mines)}")
            for multi_flag in multi_gap_flagged_mines:
                print(f"  PROP_ID {multi_flag['prop_id']}: {multi_flag['total_zero_stretches']} zero stretches")
                for stretch in multi_flag['stretches_details']:
                    print(f"    - {stretch['start_year']}-{stretch['end_year']} (length: {stretch['length']})")
        else:
            print(f"\nCriteria 5 - Multi-gap flagged mines: 0")
        
        # Drop mines
        if mines_to_drop:
            print(f"\nMines to be dropped: {len(mines_to_drop)}")
            
            # Group by reason
            criteria_1_count = len([m for m in mines_to_drop if 'Criteria 1' in m['reason']])
            criteria_3_count = len([m for m in mines_to_drop if 'Criteria 3' in m['reason']])
            
            print(f"  Criteria 1 (≥2 zeros after last production, closure > 2023): {criteria_1_count}")
            print(f"  Criteria 3 (≥4 consecutive zeros, insufficient restart): {criteria_3_count}")
            
            # Show some examples
            print(f"\nExamples of dropped mines:")
            for i, mine in enumerate(mines_to_drop[:10]):  # Show first 10
                print(f"  {mine['prop_id']}: {mine['reason']}")
            if len(mines_to_drop) > 10:
                print(f"  ... and {len(mines_to_drop) - 10} more")
            
            # Drop the mines
            indices_to_drop = [mine['idx'] for mine in mines_to_drop]
            df = df.drop(indices_to_drop).reset_index(drop=True)
        
        # Final statistics
        final_rows = len(df)
        rows_dropped = initial_rows - final_rows
        
        print(f"\n=== FINAL SUMMARY ===")
        print(f"Initial rows: {initial_rows}")
        print(f"Rows dropped: {rows_dropped}")
        print(f"Final rows: {final_rows}")
        print(f"Data retention rate: {final_rows/initial_rows*100:.1f}%")
        print(f"Closure year changes made: {len(closure_changes)}")
        print(f"Mines flagged for review (Criteria 4): {len(flagged_mines)}")
        print(f"Multi-gap flagged mines (Criteria 5): {len(multi_gap_flagged_mines)}")
        
        # Save the edited dataset
        df.to_excel(output_file, index=False)
        print(f"\nEdited dataset saved: {output_file}")
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except Exception as e:
        print(f"Error during processing: {e}")

if __name__ == "__main__":
    edit_production_dataset()

Reading dataset: Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx
Dataset loaded: 989 rows
Found 44 year columns

=== PROCESSING RESULTS ===

Criteria 2 - Closure year changes: 8
  PROP_ID 29477: Changed closure from 2019 to 2013 (10 trailing zeros)
  PROP_ID 28245: Changed closure from 2022 to 2016 (7 trailing zeros)
  PROP_ID 35734: Changed closure from 2021 to 2020 (3 trailing zeros)
  PROP_ID 24474: Changed closure from 2022 to 2001 (22 trailing zeros)
  PROP_ID 28264: Changed closure from 2004 to 2000 (20 trailing zeros)
  PROP_ID 28866: Changed closure from 2022 to 2006 (17 trailing zeros)
  PROP_ID 26968: Changed closure from 2023 to 2020 (3 trailing zeros)
  PROP_ID 30189: Changed closure from 2022 to 2004 (19 trailing zeros)

Criteria 4 - Flagged mines (≥5 non-zero values after ≥3 consecutive zeros): 135
  PROP_ID 30897: Zero stretch 2002-2006 (length: 4), 13 non-zero values after, 1 total stretches
  PROP_ID 28811: Zero stretch 2000-2014 (length: 15)

In [174]:
import pandas as pd
import numpy as np

def modify_flagged_mines_startup():
    """
    Modifies startup years for mines flagged in criteria 4 based on consecutive zero patterns.
    
    Logic:
    1) For flagged mines in criteria 4: if mine has ≥1 consecutive zeros, change startup year to first non-zero production
    2) For flagged mines in criteria 4: if mine has ≥5 consecutive zeros after startup year:
       - If closure > 2023: change startup year to first non-zero production AFTER the consecutive zeros
       - If closure ≤ 2023: drop the mine
    3) Leave flagged mines in criteria 5 unchanged
    """
    # Define file path
    input_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    
    print(f"Reading dataset: {input_file}")
    
    try:
        # Read the dataset
        df = pd.read_excel(input_file)
        print(f"Dataset loaded: {len(df)} rows")
        
        # Check required columns
        if 'PROP_ID' not in df.columns:
            print("Error: PROP_ID column not found")
            return
        if 'START_UP_YR' not in df.columns:
            print("Error: START_UP_YR column not found")
            return
        
        # Get year columns (1980-2023)
        year_columns = [str(year) for year in range(1980, 2024)]
        existing_year_columns = [col for col in year_columns if col in df.columns]
        existing_year_columns.sort(key=int)  # Ensure chronological order
        
        print(f"Found {len(existing_year_columns)} year columns")
        
        # Convert year columns to numeric
        for col in existing_year_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Convert startup year and closure year to numeric
        df['START_UP_YR'] = pd.to_numeric(df['START_UP_YR'], errors='coerce')
        df['PROJ_CLOSURE_YR'] = pd.to_numeric(df['PROJ_CLOSURE_YR'], errors='coerce')
        
        # Re-identify flagged mines (criteria 4 and 5)
        print("\nRe-identifying flagged mines...")
        
        criteria_4_mines = []
        criteria_5_mines = []
        
        for idx, row in df.iterrows():
            prop_id = row['PROP_ID']
            
            # Get production values for all years, keeping NaN as NaN
            production_values = []
            for year_col in existing_year_columns:
                value = row[year_col]
                production_values.append(value)
            
            # Find all stretches of ≥3 consecutive ACTUAL zeros
            zero_stretches = []
            current_zeros = 0
            stretch_start = None
            
            for i in range(len(production_values)):
                value = production_values[i]
                if not pd.isna(value) and value == 0:  # Only count actual zeros
                    if current_zeros == 0:
                        stretch_start = i
                    current_zeros += 1
                elif not pd.isna(value) and value > 0:  # Hit non-zero production
                    if current_zeros >= 3 and stretch_start is not None:
                        # Record this zero stretch
                        zero_stretches.append({
                            'start': stretch_start,
                            'end': i - 1,
                            'length': current_zeros
                        })
                    current_zeros = 0
                    stretch_start = None
            
            # Check for a zero stretch that extends to the end
            if current_zeros >= 3 and stretch_start is not None:
                zero_stretches.append({
                    'start': stretch_start,
                    'end': len(production_values) - 1,
                    'length': current_zeros
                })
            
            # Check if mine qualifies for criteria 4 (≥5 non-zero values after ≥3 consecutive zeros)
            mine_flagged_criteria_4 = False
            
            for stretch in zero_stretches:
                if stretch['length'] >= 3:
                    # Count NON-ZERO values after this zero stretch
                    nonzero_after_stretch = 0
                    
                    for i in range(stretch['end'] + 1, len(production_values)):
                        value = production_values[i]
                        if not pd.isna(value) and value > 0:
                            nonzero_after_stretch += 1
                    
                    if nonzero_after_stretch >= 5:
                        mine_flagged_criteria_4 = True
                        break
            
            # Add to appropriate list
            if mine_flagged_criteria_4:
                multiple_stretches_count = len([s for s in zero_stretches if s['length'] >= 3])
                
                if multiple_stretches_count > 1:
                    # Criteria 5: Multiple zero stretches
                    criteria_5_mines.append({
                        'idx': idx,
                        'prop_id': prop_id,
                        'zero_stretches': zero_stretches,
                        'has_multiple_gaps': True
                    })
                else:
                    # Criteria 4: Single zero stretch with restart
                    criteria_4_mines.append({
                        'idx': idx,
                        'prop_id': prop_id,
                        'zero_stretches': zero_stretches,
                        'has_multiple_gaps': False
                    })
        
        print(f"Identified {len(criteria_4_mines)} mines flagged in criteria 4")
        print(f"Identified {len(criteria_5_mines)} mines flagged in criteria 5")
        
        # Process criteria 4 mines - modify startup years (Step 1)
        startup_changes_step1 = []
        
        print(f"\nStep 1: Processing criteria 4 mines for startup year corrections (≥1 consecutive zeros)...")
        
        for mine in criteria_4_mines:
            idx = mine['idx']
            prop_id = mine['prop_id']
            row = df.loc[idx]
            current_startup = row['START_UP_YR']
            
            # Get production values
            production_values = []
            for year_col in existing_year_columns:
                value = row[year_col]
                production_values.append(value)
            
            # Check if mine has ≥1 consecutive zeros
            has_zero_stretch = any(stretch['length'] >= 1 for stretch in mine['zero_stretches'])
            
            if has_zero_stretch:
                # Find first year with non-zero production
                first_production_year = None
                for i, year_col in enumerate(existing_year_columns):
                    value = production_values[i]
                    if not pd.isna(value) and value > 0:
                        first_production_year = int(year_col)
                        break
                
                if first_production_year and (pd.isna(current_startup) or first_production_year != current_startup):
                    # Update startup year
                    df.at[idx, 'START_UP_YR'] = first_production_year
                    startup_changes_step1.append({
                        'prop_id': prop_id,
                        'old_startup': current_startup if not pd.isna(current_startup) else 'Empty',
                        'new_startup': first_production_year,
                        'zero_stretches_count': len(mine['zero_stretches'])
                    })
        
        # Process criteria 4 mines - Step 2: Handle ≥5 consecutive zeros after startup year
        startup_changes_step2 = []
        mines_to_drop = []
        
        print(f"\nStep 2: Processing criteria 4 mines for ≥5 consecutive zeros after startup year...")
        
        for mine in criteria_4_mines:
            idx = mine['idx']
            prop_id = mine['prop_id']
            row = df.loc[idx]
            current_startup = row['START_UP_YR']  # This might have been updated in Step 1
            closure_year = row['PROJ_CLOSURE_YR']
            
            # Skip if no valid startup or closure year
            if pd.isna(current_startup) or pd.isna(closure_year):
                continue
                
            current_startup = int(current_startup)
            closure_year = int(closure_year)
            
            # Get production values
            production_values = []
            for year_col in existing_year_columns:
                value = row[year_col]
                production_values.append(value)
            
            # Find startup year index
            startup_idx = None
            for i, year_col in enumerate(existing_year_columns):
                if int(year_col) == current_startup:
                    startup_idx = i
                    break
            
            if startup_idx is None:
                continue
            
            # Look for ANY ≥5 consecutive zeros that occur after startup year
            found_long_zeros_after_startup = False
            zeros_start_idx = None
            zeros_end_idx = None
            
            # Check all zero stretches that occur after startup year
            for stretch in mine['zero_stretches']:
                stretch_start_year = int(existing_year_columns[stretch['start']])
                
                # Check if this stretch occurs after startup year and is ≥5 zeros
                if stretch_start_year > current_startup and stretch['length'] >= 5:
                    found_long_zeros_after_startup = True
                    zeros_start_idx = stretch['start']
                    zeros_end_idx = stretch['end']
                    break  # Use the first qualifying stretch found
            
            if found_long_zeros_after_startup:
                if closure_year > 2023:
                    # Find first non-zero production AFTER the consecutive zeros
                    first_production_after_zeros = None
                    for i in range(zeros_end_idx + 1, len(production_values)):
                        value = production_values[i]
                        if not pd.isna(value) and value > 0:
                            first_production_after_zeros = int(existing_year_columns[i])
                            break
                    
                    if first_production_after_zeros:
                        # Update startup year to first production after zeros
                        df.at[idx, 'START_UP_YR'] = first_production_after_zeros
                        startup_changes_step2.append({
                            'prop_id': prop_id,
                            'old_startup': current_startup,
                            'new_startup': first_production_after_zeros,
                            'zeros_length': zeros_end_idx - zeros_start_idx + 1,
                            'closure_year': closure_year
                        })
                else:  # closure_year <= 2023
                    # Drop the mine
                    mines_to_drop.append({
                        'idx': idx,
                        'prop_id': prop_id,
                        'closure_year': closure_year,
                        'zeros_length': zeros_end_idx - zeros_start_idx + 1,
                        'zeros_start_year': int(existing_year_columns[zeros_start_idx]),
                        'zeros_end_year': int(existing_year_columns[zeros_end_idx])
                    })
        
        # Drop mines identified in Step 2
        if mines_to_drop:
            indices_to_drop = [mine['idx'] for mine in mines_to_drop]
            df = df.drop(indices_to_drop).reset_index(drop=True)
            print(f"Dropped {len(mines_to_drop)} mines with ≥5 zeros after startup and closure ≤ 2023")
        
        # Report results
        print(f"\n=== PROCESSING RESULTS ===")
        
        print(f"\nStep 1 - Criteria 4 mines processed: {len(criteria_4_mines)}")
        print(f"Step 1 - Startup year changes made: {len(startup_changes_step1)}")
        
        if startup_changes_step1:
            print(f"\nStep 1 startup year corrections:")
            for change in startup_changes_step1:
                print(f"  PROP_ID {change['prop_id']}: Changed startup from {change['old_startup']} to {change['new_startup']} ({change['zero_stretches_count']} zero stretches)")
        
        print(f"\nStep 2 - Mines with ≥5 zeros after startup: {len(startup_changes_step2) + len(mines_to_drop)}")
        print(f"Step 2 - Startup year changes (closure > 2023): {len(startup_changes_step2)}")
        print(f"Step 2 - Mines dropped (closure ≤ 2023): {len(mines_to_drop)}")
        
        if startup_changes_step2:
            print(f"\nStep 2 startup year corrections (≥5 zeros after startup, closure > 2023):")
            for change in startup_changes_step2:
                print(f"  PROP_ID {change['prop_id']}: Changed startup from {change['old_startup']} to {change['new_startup']} ({change['zeros_length']} zeros, closure: {change['closure_year']})")
        
        if mines_to_drop:
            print(f"\nStep 2 mines dropped (≥5 zeros after startup, closure ≤ 2023):")
            for mine in mines_to_drop:
                print(f"  PROP_ID {mine['prop_id']}: {mine['zeros_length']} zeros ({mine['zeros_start_year']}-{mine['zeros_end_year']}), closure: {mine['closure_year']}")
        
        print(f"\nCriteria 5 mines: {len(criteria_5_mines)} (left unchanged)")
        if criteria_5_mines:
            print("Criteria 5 mines (multiple gaps, no changes made):")
            for mine in criteria_5_mines[:10]:  # Show first 10
                print(f"  PROP_ID {mine['prop_id']}: {len(mine['zero_stretches'])} zero stretches")
            if len(criteria_5_mines) > 10:
                print(f"  ... and {len(criteria_5_mines) - 10} more")
        
        # Save the modified dataset
        df.to_excel(output_file, index=False)
        
        print(f"\n=== FINAL SUMMARY ===")
        print(f"Total criteria 4 mines: {len(criteria_4_mines)}")
        print(f"Step 1 startup years corrected: {len(startup_changes_step1)}")
        print(f"Step 2 startup years corrected: {len(startup_changes_step2)}")
        print(f"Step 2 mines dropped: {len(mines_to_drop)}")
        print(f"Criteria 5 mines (unchanged): {len(criteria_5_mines)}")
        print(f"Final dataset rows: {len(df)}")
        print(f"Modified dataset saved: {output_file}")
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except Exception as e:
        print(f"Error during processing: {e}")

if __name__ == "__main__":
    modify_flagged_mines_startup()

Reading dataset: Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx
Dataset loaded: 955 rows
Found 44 year columns

Re-identifying flagged mines...
Identified 131 mines flagged in criteria 4
Identified 6 mines flagged in criteria 5

Step 1: Processing criteria 4 mines for startup year corrections (≥1 consecutive zeros)...

Step 2: Processing criteria 4 mines for ≥5 consecutive zeros after startup year...
Dropped 1 mines with ≥5 zeros after startup and closure ≤ 2023

=== PROCESSING RESULTS ===

Step 1 - Criteria 4 mines processed: 131
Step 1 - Startup year changes made: 24

Step 1 startup year corrections:
  PROP_ID 30897: Changed startup from 2001 to 2007 (1 zero stretches)
  PROP_ID 27467: Changed startup from 1990 to 2016 (1 zero stretches)
  PROP_ID 80829: Changed startup from 1950 to 2016 (1 zero stretches)
  PROP_ID 27163: Changed startup from 1969 to 1992 (1 zero stretches)
  PROP_ID 31323: Changed startup from 2004 to 2006 (1 zero stretches)
  PROP_ID 35

Merge with Maus Spatial Data
1) Merge the cluster id of each mine into the dataset based on the mines PROP_ID and the column 'id_data_source' in the spatial data xlsx file
2) drop any mines that have no cluster id
3) if there are more than 1 mine sharing the same cluster id and they are different mine types (open pit and underground), drop them because you cannot allocate area based on cumulative production between underground and open pit mines

In [176]:
import pandas as pd

def merge_cluster_data(csv_path, excel_path):
    """
    Merge cluster IDs from a CSV file into an Excel file based on property IDs,
    removing rows without assigned cluster IDs, and save back to the original Excel file.
    The id_cluster column will be added as the rightmost column.
    
    Parameters:
    -----------
    csv_path : str
        Path to the CSV file containing cluster data
    excel_path : str
        Path to the Excel file to be updated
    
    Returns:
    --------
    DataFrame
        The merged and filtered DataFrame
    """
    # Read the CSV file with cluster data
    print(f"Reading cluster data from {csv_path}...")
    cluster_df = pd.read_csv(csv_path)
    print(f"Found {len(cluster_df)} rows in cluster data")
    
    # Read the Excel file
    print(f"Reading Excel file from {excel_path}...")
    excel_df = pd.read_excel(excel_path)
    print(f"Found {len(excel_df)} rows in Excel file")
    
    # Verify column names exist
    if 'id_data_source' not in cluster_df.columns:
        raise ValueError(f"Column 'id_data_source' not found in the CSV file. Available columns: {cluster_df.columns.tolist()}")
    if 'id_cluster' not in cluster_df.columns:
        raise ValueError(f"Column 'id_cluster' not found in the CSV file. Available columns: {cluster_df.columns.tolist()}")
    if 'PROP_ID' not in excel_df.columns:
        raise ValueError(f"Column 'PROP_ID' not found in the Excel file. Available columns: {excel_df.columns.tolist()}")
    
    # Convert both ID columns to strings to ensure proper matching
    cluster_df['id_data_source'] = cluster_df['id_data_source'].astype(str)
    excel_df['PROP_ID'] = excel_df['PROP_ID'].astype(str)
    
    # Create a subset of the cluster data with only the necessary columns
    cluster_subset = cluster_df[['id_data_source', 'id_cluster']].copy()
    
    # Rename column to match Excel for merging
    cluster_subset = cluster_subset.rename(columns={'id_data_source': 'PROP_ID'})
    
    # Merge the data
    print("Merging data...")
    merged_df = pd.merge(
        excel_df,
        cluster_subset,
        on='PROP_ID',
        how='left'
    )
    
    # If id_cluster already exists in the original data, this will create a duplicate
    # Ensure we have only one id_cluster column
    if 'id_cluster_x' in merged_df.columns and 'id_cluster_y' in merged_df.columns:
        # This happens when id_cluster already existed in excel_df
        # Keep the newly merged column and drop the original
        merged_df = merged_df.drop(columns=['id_cluster_x'])
        merged_df = merged_df.rename(columns={'id_cluster_y': 'id_cluster'})
    
    # Ensure id_cluster is the last column
    if 'id_cluster' in merged_df.columns:
        # Get all columns except id_cluster
        other_cols = [col for col in merged_df.columns if col != 'id_cluster']
        # Reorder columns to put id_cluster at the end
        merged_df = merged_df[other_cols + ['id_cluster']]
    
    # Count rows before filtering
    total_rows = len(merged_df)
    
    # Filter out rows without an id_cluster
    filtered_df = merged_df.dropna(subset=['id_cluster'])
    
    # Count removed rows
    removed_rows = total_rows - len(filtered_df)
    print(f"Removed {removed_rows} rows with no id_cluster assigned ({removed_rows/total_rows:.1%} of total)")
    
    # Save back to the original Excel file
    print(f"Saving merged data back to {excel_path}...")
    filtered_df.to_excel(excel_path, index=False)
    
    print(f"Done! Final dataset has {len(filtered_df)} rows.")
    return filtered_df

# Example usage
if __name__ == "__main__":
    csv_path = "Data Input/Maus_Spa_Data/cluster_points_concordance.csv"
    excel_path = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    
    try:
        merge_cluster_data(csv_path, excel_path)
    except Exception as e:
        print(f"Error: {e}")

Reading cluster data from Data Input/Maus_Spa_Data/cluster_points_concordance.csv...
Found 42799 rows in cluster data
Reading Excel file from Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx...
Found 763 rows in Excel file
Merging data...
Removed 0 rows with no id_cluster assigned (0.0% of total)
Saving merged data back to Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx...
Done! Final dataset has 763 rows.


In [177]:
import pandas as pd

def analyze_and_clean_id_clusters():
    """
    Analyze the number of mines (PROP_ID) that share the same id_cluster,
    check for open pit vs underground mines sharing the same cluster,
    and remove problematic clusters from the dataset.
    """
    
    # Read the Excel file
    file_path = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    #file_path = "Data Output/Final_Dataset.xlsx"
    print(f"Reading data from {file_path}...")
    
    try:
        df = pd.read_excel(file_path)
        print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return
    
    # Check if required columns exist
    required_columns = ['PROP_ID', 'id_cluster', 'MINE_TYPE1']
    optional_columns = ['PROP_NAME', 'COUNTRY', 'PRIMARY_COMMODITY']
    
    missing_required = [col for col in required_columns if col not in df.columns]
    if missing_required:
        print(f"Missing required columns: {missing_required}")
        return
    
    missing_optional = [col for col in optional_columns if col not in df.columns]
    if missing_optional:
        print(f"Warning: Missing optional columns for detailed info: {missing_optional}")
        print("Will proceed with available columns only.")
    
    available_detail_columns = [col for col in optional_columns if col in df.columns]
    
    # Remove rows with missing values in key columns
    original_count = len(df)
    df_clean = df.dropna(subset=required_columns).copy()
    print(f"After removing rows with missing values: {len(df_clean)} rows (removed {original_count - len(df_clean)} rows)")
    
    print("\n" + "="*60)
    print("ID CLUSTER ANALYSIS RESULTS")
    print("="*60)
    
    # 1. Basic statistics about clusters
    total_clusters = df_clean['id_cluster'].nunique()
    total_mines = df_clean['PROP_ID'].nunique()
    
    print(f"\nBasic Statistics:")
    print(f"- Total unique ID clusters: {total_clusters}")
    print(f"- Total unique mines (PROP_ID): {total_mines}")
    
    # 2. Analyze mines per cluster
    cluster_mine_counts = df_clean.groupby('id_cluster')['PROP_ID'].nunique().reset_index()
    cluster_mine_counts.columns = ['id_cluster', 'mine_count']
    
    print(f"\nMines per Cluster Distribution:")
    mine_count_distribution = cluster_mine_counts['mine_count'].value_counts().sort_index()
    for count, frequency in mine_count_distribution.items():
        print(f"- {frequency} clusters have {count} mine(s)")
    
    # 3. Find clusters with multiple mines
    multi_mine_clusters = cluster_mine_counts[cluster_mine_counts['mine_count'] > 1]
    print(f"\nClusters with multiple mines: {len(multi_mine_clusters)}")
    
    if len(multi_mine_clusters) > 0:
        print("\nTop 10 clusters with most mines:")
        top_clusters = multi_mine_clusters.nlargest(10, 'mine_count')
        for _, row in top_clusters.iterrows():
            print(f"- Cluster {row['id_cluster']}: {row['mine_count']} mines")
    
    # 4. Analyze mine types within clusters
    print(f"\n" + "="*60)
    print("MINE TYPE ANALYSIS WITHIN CLUSTERS")
    print("="*60)
    
    # Get unique mine types
    mine_types = df_clean['MINE_TYPE1'].value_counts()
    print(f"\nOverall mine type distribution:")
    for mine_type, count in mine_types.items():
        print(f"- {mine_type}: {count} mines")
    
    # Analyze mine types per cluster
    cluster_mine_types = df_clean.groupby('id_cluster').agg({
        'PROP_ID': 'nunique',
        'MINE_TYPE1': lambda x: list(x.unique())
    }).reset_index()
    cluster_mine_types.columns = ['id_cluster', 'mine_count', 'mine_types']
    
    # 5. Find clusters with mixed mine types
    print(f"\nAnalyzing clusters with mixed mine types...")
    
    mixed_type_clusters = []
    clusters_to_remove = []
    
    for _, row in cluster_mine_types.iterrows():
        if len(row['mine_types']) > 1:
            mixed_type_clusters.append({
                'cluster': row['id_cluster'],
                'mine_count': row['mine_count'],
                'mine_types': row['mine_types']
            })
            # Add this cluster to removal list
            clusters_to_remove.append(row['id_cluster'])
    
    print(f"\nClusters with mixed mine types: {len(mixed_type_clusters)}")
    
    if len(mixed_type_clusters) > 0:
        print(f"\nDetailed breakdown of mixed-type clusters:")
        for cluster_info in mixed_type_clusters:
            types_str = ", ".join(cluster_info['mine_types'])
            print(f"- Cluster {cluster_info['cluster']}: {cluster_info['mine_count']} mines ({types_str})")
        
        # 6. Specifically check for Open Pit + Underground combinations
        open_underground_clusters = []
        for cluster_info in mixed_type_clusters:
            types = cluster_info['mine_types']
            if 'Open Pit' in types and 'Underground' in types:
                open_underground_clusters.append(cluster_info)
        
        print(f"\n" + "-"*50)
        print(f"CLUSTERS WITH BOTH OPEN PIT AND UNDERGROUND MINES")
        print(f"-"*50)
        print(f"Found {len(open_underground_clusters)} clusters with both Open Pit and Underground mines:")
        
        if len(open_underground_clusters) > 0:
            for cluster_info in open_underground_clusters:
                cluster_id = cluster_info['cluster']
                print(f"\nCluster {cluster_id}:")
                
                # Get detailed breakdown for this cluster
                cluster_data = df_clean[df_clean['id_cluster'] == cluster_id]
                mine_type_breakdown = cluster_data.groupby('MINE_TYPE1')['PROP_ID'].nunique()
                
                for mine_type, count in mine_type_breakdown.items():
                    print(f"  - {mine_type}: {count} mines")
                
                # Show detailed information for all mines in this cluster
                print(f"\n  Detailed mine information for Cluster {cluster_id}:")
                print(f"  {'-'*80}")
                
                # Sort by mine type for better organization
                cluster_mines = cluster_data.sort_values(['MINE_TYPE1', 'PROP_ID'])
                
                for _, mine_row in cluster_mines.iterrows():
                    mine_id = mine_row['PROP_ID']
                    mine_type = mine_row['MINE_TYPE1']
                    
                    # Build detail string with available information
                    details = []
                    
                    if 'PROP_NAME' in available_detail_columns:
                        prop_name = mine_row['PROP_NAME'] if pd.notna(mine_row['PROP_NAME']) else 'N/A'
                        details.append(f"Name: {prop_name}")
                    
                    if 'COUNTRY' in available_detail_columns:
                        country = mine_row['COUNTRY'] if pd.notna(mine_row['COUNTRY']) else 'N/A'
                        details.append(f"Country: {country}")
                    
                    if 'PRIMARY_COMMODITY' in available_detail_columns:
                        commodity = mine_row['PRIMARY_COMMODITY'] if pd.notna(mine_row['PRIMARY_COMMODITY']) else 'N/A'
                        details.append(f"Commodity: {commodity}")
                    
                    detail_string = " | ".join(details) if details else "No additional details available"
                    
                    print(f"    {mine_type} - PROP_ID: {mine_id}")
                    print(f"      {detail_string}")
                
                print(f"  {'-'*80}")
        else:
            print("No clusters found with both Open Pit and Underground mines.")
    else:
        print("No clusters found with mixed mine types - all clusters contain only one type of mine.")
    
    # 7. Remove problematic clusters from the dataset
    print(f"\n" + "="*60)
    print("DATA CLEANING - REMOVING MIXED-TYPE CLUSTERS")
    print("="*60)
    
    if len(clusters_to_remove) > 0:
        print(f"\nRemoving {len(clusters_to_remove)} clusters with mixed mine types...")
        
        # Count mines to be removed
        mines_to_remove = df_clean[df_clean['id_cluster'].isin(clusters_to_remove)]['PROP_ID'].nunique()
        rows_to_remove = len(df_clean[df_clean['id_cluster'].isin(clusters_to_remove)])
        
        print(f"- Clusters to remove: {clusters_to_remove}")
        print(f"- Unique mines being removed: {mines_to_remove}")
        print(f"- Total rows being removed: {rows_to_remove}")
        
        # Remove the problematic clusters from the original dataframe
        df_cleaned = df[~df['id_cluster'].isin(clusters_to_remove)].copy()
        
        print(f"\nDataset size before cleaning: {len(df)} rows")
        print(f"Dataset size after cleaning: {len(df_cleaned)} rows")
        print(f"Rows removed: {len(df) - len(df_cleaned)}")
        
        # Save the cleaned dataset
        try:
            df_cleaned.to_excel(file_path, index=False)
            print(f"\nCleaned dataset saved back to: {file_path}")
            print("✓ Data cleaning completed successfully!")
        except Exception as e:
            print(f"\nError saving cleaned dataset: {e}")
            print("The analysis was completed but the file could not be saved.")
    else:
        print("\nNo mixed-type clusters found - no data cleaning required.")
        print("Dataset remains unchanged.")
    
    # 8. Summary statistics
    print(f"\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    
    single_mine_clusters = len(cluster_mine_counts[cluster_mine_counts['mine_count'] == 1])
    multi_mine_clusters_count = len(cluster_mine_counts[cluster_mine_counts['mine_count'] > 1])
    
    print(f"- Total clusters (before cleaning): {total_clusters}")
    print(f"- Clusters with single mine: {single_mine_clusters} ({single_mine_clusters/total_clusters*100:.1f}%)")
    print(f"- Clusters with multiple mines: {multi_mine_clusters_count} ({multi_mine_clusters_count/total_clusters*100:.1f}%)")
    print(f"- Clusters with mixed mine types: {len(mixed_type_clusters)}")
    if len(mixed_type_clusters) > 0:
        print(f"- Clusters with Open Pit + Underground: {len(open_underground_clusters)}")
        print(f"- Clusters removed from dataset: {len(clusters_to_remove)}")

if __name__ == "__main__":
    analyze_and_clean_id_clusters()

Reading data from Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx...
Dataset loaded with 763 rows and 61 columns
Will proceed with available columns only.
After removing rows with missing values: 763 rows (removed 0 rows)

ID CLUSTER ANALYSIS RESULTS

Basic Statistics:
- Total unique ID clusters: 640
- Total unique mines (PROP_ID): 763

Mines per Cluster Distribution:
- 567 clusters have 1 mine(s)
- 53 clusters have 2 mine(s)
- 9 clusters have 3 mine(s)
- 7 clusters have 4 mine(s)
- 2 clusters have 5 mine(s)
- 1 clusters have 6 mine(s)
- 1 clusters have 19 mine(s)

Clusters with multiple mines: 73

Top 10 clusters with most mines:
- Cluster H0027425: 19 mines
- Cluster H0027430: 6 mines
- Cluster H0022218: 5 mines
- Cluster H0043023: 5 mines
- Cluster H0027205: 4 mines
- Cluster H0035427: 4 mines
- Cluster H0035565: 4 mines
- Cluster H0040706: 4 mines
- Cluster H0042323: 4 mines
- Cluster H0045007: 4 mines

MINE TYPE ANALYSIS WITHIN CLUSTERS

Overall mine typ

Conversion to long format for cumulative production use
1) convert to long format
2) delete all rows before start up year and after closure year if applicable to reduce number of empty cells

In [183]:
import os
import pandas as pd
import re

def convert_to_long_format(input_file, output_file=None):
    """
    Convert a wide-format Excel file to long format by transforming year columns
    into a single Year column with corresponding values for each mine property.
    For coal commodities with the same PROP_ID, groups all thermal coal records together
    and all metallurgical coal records together, with each group ordered by year.
   
    Args:
        input_file (str): Path to the wide-format Excel file
        output_file (str, optional): Path for saving the long-format output file.
                                    If None, will use input_file with "_long" appended.
    """
    # Set default output file if not provided
    if output_file is None:
        base_name, ext = os.path.splitext(input_file)
        output_file = f"{base_name}_long{ext}"
   
    # Create output directory if it doesn't exist
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
   
    print(f"Reading file: {input_file}")
   
    try:
        # Read the Excel file
        df = pd.read_excel(input_file)
       
        # Check if PROP_ID column exists
        if 'PROP_ID' not in df.columns:
            print("Error: PROP_ID column not found in the input file.")
            return
       
        print(f"Original data shape: {df.shape}")
       
        # Identify year columns (assuming they are named like "1980", "1981", etc.)
        year_pattern = re.compile(r'^(19|20)\d{2}$')  # Match years from 1900-2099
        year_columns = [col for col in df.columns if isinstance(col, str) and year_pattern.match(col)]
       
        if not year_columns:
            # Try to identify if years might be stored as integers or floats
            year_columns = [col for col in df.columns if isinstance(col, (int, float)) and 1900 <= col <= 2099]
       
        if not year_columns:
            print("No year columns found in the data. Please check the column names.")
            return
       
        print(f"Found {len(year_columns)} year columns: {year_columns[:5]}..."
              f"{year_columns[-5:]}" if len(year_columns) > 10 else year_columns)
       
        # Identify ID columns and attribute columns
        # Assuming all non-year columns are attributes to keep
        id_and_attribute_columns = [col for col in df.columns if col not in year_columns]
       
        print("Converting to long format...")
       
        # Use pandas melt to convert to long format
        long_df = pd.melt(
            df,
            id_vars=id_and_attribute_columns,
            value_vars=year_columns,
            var_name='Year',
            value_name='Production'
        )
       
        # Convert Year column to integer if needed
        long_df['Year'] = pd.to_numeric(long_df['Year'], errors='coerce').astype('Int64')
        
        # Check if PRIMARY_COMMODITY column exists
        if 'PRIMARY_COMMODITY' in long_df.columns:
            print("Processing coal commodities...")
            
            # Create a coal type indicator with proper sorting order
            def get_coal_type(commodity):
                if not isinstance(commodity, str):
                    return 2  # Non-coal or non-string types last
                
                commodity = commodity.lower()
                if 'coal (thermal)' in commodity:
                    return 0  # Thermal coal first
                elif 'coal (metallurgical)' in commodity:
                    return 1  # Metallurgical coal second
                else:
                    return 2  # Non-coal types last
            
            # Add a temporary coal type indicator column
            long_df['coal_type'] = long_df['PRIMARY_COMMODITY'].apply(get_coal_type)
            
            # Sort first by PROP_ID to group all records for the same property
            # Then by coal_type to group all thermal coal before metallurgical
            # Then by Year to ensure chronological order within each coal type
            print("Sorting data by PROP_ID, coal type, and Year...")
            long_df = long_df.sort_values(by=['PROP_ID', 'coal_type', 'Year'])
            
            # Remove the temporary column
            long_df = long_df.drop(columns=['coal_type'])
        else:
            # If PRIMARY_COMMODITY doesn't exist, just sort by PROP_ID and Year
            print("PRIMARY_COMMODITY column not found. Sorting by PROP_ID and Year only...")
            long_df = long_df.sort_values(by=['PROP_ID', 'Year'])
       
        print(f"Long format data shape: {long_df.shape}")
       
        # Save to Excel
        print(f"Saving to: {output_file}")
        long_df.to_excel(output_file, index=False)
       
        print("Conversion completed successfully!")
        return long_df
       
    except Exception as e:
        print(f"Error processing file: {e}")

# Example usage
if __name__ == "__main__":
    input_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
    output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_long.xlsx"
    convert_to_long_format(input_file, output_file)

Reading file: Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx
Original data shape: (723, 61)
Found 44 year columns: ['1980', '1981', '1982', '1983', '1984']...['2019', '2020', '2021', '2022', '2023']
Converting to long format...
Processing coal commodities...
Sorting data by PROP_ID, coal type, and Year...
Long format data shape: (31812, 19)
Saving to: Data Output/Commodity Production/Commodity_Production-1980_2023_long.xlsx
Conversion completed successfully!


In [184]:
import pandas as pd
import os
from datetime import datetime

def filter_mines_by_operation_years(input_file, create_backup=True):
    """
    Filter the commodity production dataset to keep only rows where:
    1. Year >= START_UP_YR for each mine, and
    2. Year <= PROJ_CLOSURE_YR (if PROJ_CLOSURE_YR is not null)
    
    Args:
        input_file (str): Path to the Excel file to filter
        create_backup (bool): Whether to create a backup of the original file
    
    Returns:
        DataFrame: The filtered data
    """
    try:
        print(f"Loading data from {input_file}...")
        # Read the Excel file
        df = pd.read_excel(input_file)
        
        # Check if required columns exist
        required_columns = ['Year', 'START_UP_YR', 'PROP_ID', 'PROJ_CLOSURE_YR']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
        
        # Get the initial row count and unique properties
        initial_count = len(df)
        initial_properties = df['PROP_ID'].nunique()
        print(f"Initial dataset: {initial_count} rows, {initial_properties} unique properties")
        
        # First filter: Keep only rows where Year >= START_UP_YR
        filtered_startup = df[df['Year'] >= df['START_UP_YR']]
        startup_filtered_count = len(filtered_startup)
        startup_removed = initial_count - startup_filtered_count
        
        print(f"After START_UP_YR filter: {startup_filtered_count} rows")
        print(f"Removed {startup_removed} rows ({startup_removed/initial_count:.2%}) that were before START_UP_YR")
        
        # Second filter: Keep only rows where Year <= PROJ_CLOSURE_YR (if PROJ_CLOSURE_YR is not null)
        # Create a mask where either PROJ_CLOSURE_YR is NaN (meaning no closure date) 
        # or Year <= PROJ_CLOSURE_YR
        closure_mask = filtered_startup['PROJ_CLOSURE_YR'].isna() | (filtered_startup['Year'] <= filtered_startup['PROJ_CLOSURE_YR'])
        filtered_df = filtered_startup[closure_mask]
        
        # Get the filtered counts
        filtered_count = len(filtered_df)
        filtered_properties = filtered_df['PROP_ID'].nunique()
        closure_removed = startup_filtered_count - filtered_count
        total_removed = initial_count - filtered_count
        
        print(f"After PROJ_CLOSURE_YR filter: {filtered_count} rows")
        print(f"Removed {closure_removed} rows ({closure_removed/startup_filtered_count:.2%}) that were after PROJ_CLOSURE_YR")
        print(f"Total filtered dataset: {filtered_count} rows, {filtered_properties} unique properties")
        print(f"Total removed: {total_removed} rows ({total_removed/initial_count:.2%} of original data)")
        
        # Save the filtered dataframe to a new file
        output_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_long.xlsx"
        print(f"Saving filtered data to {output_file}...")
        filtered_df.to_excel(output_file, index=False)
        print("Done!")
        
        # Generate a summary of data before and after filtering
        summary = pd.DataFrame({
            'Total Rows': [initial_count, startup_filtered_count, filtered_count, total_removed],
            'Unique Properties': [
                initial_properties, 
                filtered_startup['PROP_ID'].nunique(), 
                filtered_properties,
                initial_properties - filtered_properties
            ],
            'Description': [
                'Original data',
                'After START_UP_YR filter', 
                'After both filters', 
                'Total rows removed'
            ],
            'Percent of Original': [
                '100%',
                f"{startup_filtered_count/initial_count:.2%}",
                f"{filtered_count/initial_count:.2%}",
                f"{total_removed/initial_count:.2%}"
            ]
        })
        
        print("\nSummary Statistics:")
        print(summary)
        
        return filtered_df
        
    except Exception as e:
        print(f"Error: {e}")
        return None

if __name__ == "__main__":
    # File path for your data
    input_file = "Data Output/Commodity Production/Commodity_Production-1980_2023_long.xlsx"
    
    # Run the filtering function
    filtered_data = filter_mines_by_operation_years(input_file)
    
    # Only display sample if data was successfully filtered
    if filtered_data is not None:
        print("\nSample of filtered data (first 5 rows):")
        print(filtered_data.head())

Loading data from Data Output/Commodity Production/Commodity_Production-1980_2023_long.xlsx...
Initial dataset: 31812 rows, 723 unique properties
After START_UP_YR filter: 19042 rows
Removed 12770 rows (40.14%) that were before START_UP_YR
After PROJ_CLOSURE_YR filter: 18493 rows
Removed 549 rows (2.88%) that were after PROJ_CLOSURE_YR
Total filtered dataset: 18493 rows, 723 unique properties
Total removed: 13319 rows (41.87% of original data)
Saving filtered data to Data Output/Commodity Production/Commodity_Production-1980_2023_long.xlsx...
Done!

Summary Statistics:
   Total Rows  Unique Properties               Description Percent of Original
0       31812                723             Original data                100%
1       19042                723  After START_UP_YR filter              59.86%
2       18493                723        After both filters              58.13%
3       13319                  0        Total rows removed              41.87%

Sample of filtered data (fir